In [34]:
# coding=utf-8
# Copyright 2018 The Google AI Language Team Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
r"""Utilities to simplify the canonical NQ data.

The canonical NQ data contains the HTML of each Wikipedia page along with a
sequence of tokens on that page, each of which is indexed into the HTML.

Many users will not want to use the HTML at all, and this file provides
utilities to extract only the text into a new record of the form:

  {
    "example_id": 3902,
    "document_url": "http://wikipedia.org/en/strings"
    "question_text": "what is a string",
    "document_text": "<P> A string is a list of characters in order . </P>",
    "annotations": [{
      "long_answer": { "start_token": 0, "end_token": 12 },
      "short_answers": [{ "start_token": 5, "end_token": 8 }],
      "yes_no_answer": "NONE",
    }],
    "long_answer_candidates": [
      {"start_token": 0, "end_token": 12, "top_level": True}
    ]
  }

which leads to a much smaller training set (4.4Gb instead of 41Gb).

In this representation, the [start, end) indices are into the blank separated
sequence of tokens. So, answer spans can be extracted using the following
snippet:

  " ".join(example["document_text"].split(" ")[`start_token`:`end_token`]).

WARNING: Use `split(" ")` instead of `split()` to avoid complications from
  characters such as `\u180e` which may or may not be recognized as a whitespace
  character depending on your python version.

To avoid complications at test time, we do not provide a simplified version
of the development data, and there is no simplified version of the hidden test
set. If you rely on the simplified data, then you must call the
`simplify_nq_example` function below on every example that is passed in at test
time.
"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import re


def get_nq_tokens(simplified_nq_example):
  """Returns list of blank separated tokens."""

  if "document_text" not in simplified_nq_example:
    raise ValueError("`get_nq_tokens` should be called on a simplified NQ"
                     "example that contains the `document_text` field.")

  return simplified_nq_example["document_text"].split(" ")


def simplify_nq_example(nq_example):
  r"""Returns dictionary with blank separated tokens in `document_text` field.

  Removes byte offsets from annotations, and removes `document_html` and
  `document_tokens` fields. All annotations in the ouput are represented as
  [start_token, end_token) offsets into the blank separated tokens in the
  `document_text` field.

  WARNING: Tokens are separated by a single blank character. Do not split on
    arbitrary whitespace since different implementations have different
    treatments of some unicode characters such as \u180e.

  Args:
    nq_example: Dictionary containing original NQ example fields.

  Returns:
    Dictionary containing `document_text` field, not containing
    `document_tokens` or `document_html`, and with all annotations represented
    as [`start_token`, `end_token`) offsets into the space separated sequence.
  """

  def _clean_token(token):
    """Returns token in which blanks are replaced with underscores.

    HTML table cell openers may contain blanks if they span multiple columns.
    There are also a very few unicode characters that are prepended with blanks.

    Args:
      token: Dictionary representation of token in original NQ format.

    Returns:
      String token.
    """
    return re.sub(u" ", "_", token["token"])

  text = " ".join([_clean_token(t) for t in nq_example["document_tokens"]])

  def _remove_html_byte_offsets(span):
    if "start_byte" in span:
      del span["start_byte"]

    if "end_byte" in span:
      del span["end_byte"]

    return span

  def _clean_annotation(annotation):
    annotation["long_answer"] = _remove_html_byte_offsets(
        annotation["long_answer"])
    annotation["short_answers"] = [
        _remove_html_byte_offsets(sa) for sa in annotation["short_answers"]
    ]
    return annotation

  simplified_nq_example = {
      "question_text": nq_example["question_text"],
      "example_id": nq_example["example_id"],
      "document_url": nq_example["document_url"],
      "document_text": text,
      "long_answer_candidates": [
          _remove_html_byte_offsets(c)
          for c in nq_example["long_answer_candidates"]
      ],
      "annotations": [_clean_annotation(a) for a in nq_example["annotations"]]
  }

  if len(get_nq_tokens(simplified_nq_example)) != len(
      nq_example["document_tokens"]):
    raise ValueError("Incorrect number of tokens.")

  return simplified_nq_example

In [85]:
from datasets import load_dataset
from pathlib import Path
from pprint import pprint
from datasets import load_dataset
import pandas as pd
import os
from dotenv import load_dotenv
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import pandas
import openai
from openai import APIError
import os
import json
import re
import numpy as np
from sklearn.cluster import KMeans
from pprint import pprint
from pathlib import Path
import tiktoken


# Load the .env file
load_dotenv()

# Access the variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai.api_key = OPENAI_API_KEY



In [4]:


dir = Path("~/Datasets/SRBendding").expanduser()
ds = load_dataset("google-research-datasets/natural_questions", "dev", cache_dir=dir)

Resolving data files:   0%|          | 0/287 [00:00<?, ?it/s]

Generating validation split:   0%|          | 0/7830 [00:00<?, ? examples/s]

In [5]:
ds

DatasetDict({
    validation: Dataset({
        features: ['id', 'document', 'question', 'long_answer_candidates', 'annotations'],
        num_rows: 7830
    })
})

In [14]:
vali = ds['validation']
vali

Dataset({
    features: ['id', 'document', 'question', 'long_answer_candidates', 'annotations'],
    num_rows: 7830
})

In [65]:
# vali[0]['document']['tokens']['start_byte']

In [66]:
# vali[0]['long_answer_candidates']["start_byte"]

In [70]:
vali[0]['annotations']['long_answer']

[{'start_token': 932,
  'end_token': 1016,
  'start_byte': 87119,
  'end_byte': 88080,
  'candidate_index': 66},
 {'start_token': -1,
  'end_token': -1,
  'start_byte': -1,
  'end_byte': -1,
  'candidate_index': -1},
 {'start_token': -1,
  'end_token': -1,
  'start_byte': -1,
  'end_byte': -1,
  'candidate_index': -1},
 {'start_token': 3301,
  'end_token': 3669,
  'start_byte': 118430,
  'end_byte': 121038,
  'candidate_index': 80},
 {'start_token': -1,
  'end_token': -1,
  'start_byte': -1,
  'end_byte': -1,
  'candidate_index': -1}]

In [67]:
# simplify_nq_example(vali[0])

In [76]:
result = []
for i in range(len(vali)):
    info = vali[i]
    id = info['id']
    context = ""
    # start_byte = info['long_answer_candidates']["start_byte"][0]
    # end_byte = info['long_answer_candidates']["end_byte"][0]
    start_byte = -1
    end_byte = -1
    long_answers = info['annotations']['long_answer']
    for j in range(len(long_answers)):
        if long_answers[j]["start_byte"] != -1:
            start_byte = long_answers[j]["start_byte"]
            end_byte = long_answers[j]["end_byte"]
            break
                
    token_element = info['document']['tokens']
    for j in range(len(token_element['token'])):
        if not token_element['is_html'][j] and start_byte <= token_element['start_byte'][j] <= end_byte:
            context += (token_element['token'][j]).strip() + " "
    context = context.strip()
    question = info['question']['text']
    current = {
        "query_id": id,
        "query": question,
        "passage_text": context,

    }
    result.append(current)

pprint(len(result))



7830


In [80]:
pprint(result[:5])

[{'passage_text': 'Through the work of Max Planck , Albert Einstein , Louis de '
                  'Broglie , Arthur Compton , Niels Bohr and many others , '
                  'current scientific theory holds that all particles also '
                  'have a wave nature ( and vice versa ) . This phenomenon has '
                  'been verified not only for elementary particles , but also '
                  'for compound particles like atoms and even molecules . For '
                  'macroscopic particles , because of their extremely short '
                  'wavelengths , wave properties usually can not be detected .',
  'query': 'who proposed that electrons behave like waves and particles',
  'query_id': '-5501481664893105662'},
 {'passage_text': 'The United States Senate consists of 100 members , two from '
                  'each of the 50 states . Below is a list of the current U.S. '
                  'Senators , sitting in the 115th United States Congress .',
  'query': '

In [81]:
def load_data_natural(dataset_name:str = "google-research-datasets/natural_questions"):
    dir = Path("~/Datasets/SRBendding").expanduser()
    data = load_dataset(dataset_name, "dev", cache_dir=dir)  
    validation_dataset = data['validation']

    result = []
    for i in range(len(validation_dataset) - 7820):
        record = validation_dataset[i]
        id = record['id']
        start_byte, end_byte = get_start_and_end_byte(record)
        context = make_context(record, start_byte, end_byte)
        question = record['question']['text']
        current = {
            "query_id": id,
            "query": question,
            "passage_text": [context],

        }
        result.append(current)

    return result

def make_context(info, start_byte, end_byte):
    context = ""    
    token_element = info['document']['tokens']
    for j in range(len(token_element['token'])):
        if not token_element['is_html'][j] and start_byte <= token_element['start_byte'][j] <= end_byte:
            context += (token_element['token'][j]).strip() + " "
    context = context.strip()
    return context

def get_start_and_end_byte(info):
    start_byte = -1
    end_byte = -1
    long_answers = info['annotations']['long_answer']
    for j in range(len(long_answers)):
        if long_answers[j]["start_byte"] != -1:
            start_byte = long_answers[j]["start_byte"]
            end_byte = long_answers[j]["end_byte"]
            break
    return start_byte,end_byte

In [82]:
res = load_data_natural()


Resolving data files:   0%|          | 0/287 [00:00<?, ?it/s]

In [84]:
res

[{'query_id': '-5501481664893105662',
  'query': 'who proposed that electrons behave like waves and particles',
  'passage_text': ['Through the work of Max Planck , Albert Einstein , Louis de Broglie , Arthur Compton , Niels Bohr and many others , current scientific theory holds that all particles also have a wave nature ( and vice versa ) . This phenomenon has been verified not only for elementary particles , but also for compound particles like atoms and even molecules . For macroscopic particles , because of their extremely short wavelengths , wave properties usually can not be detected .']},
 {'query_id': '8594030243394572667',
  'query': 'how many senators are there in the us senate',
  'passage_text': ['The United States Senate consists of 100 members , two from each of the 50 states . Below is a list of the current U.S. Senators , sitting in the 115th United States Congress .']},
 {'query_id': '-716176363900512091',
  'query': 'phase change from gas to solid is called',
  'passa

In [78]:
count = 0
for res in result:
    if len(res['passage_text']) < 10:
        count += 1
        print(res['passage_text'])
print(count)

In [159]:
loaded_table = pq.read_table('datasets/msmarco.parquet')
df = loaded_table.to_pandas()

In [170]:
for q in df['passage_text']:
    print(q)

['Ime Nigel je latinskog porekla. Na latinskom jeziku značenje imena Nigel je: Tamno. Američko značenje: Ime Nigel je američko ime. U Americi'
 'Ime Nigel je irskog porekla. Na irskom jeziku značenje imena Nigel je: Šampion. Dinastija irskih kraljeva osnovana je od strane Nialla od Devet Zarobljenika. Englesko značenje: Ime Nigel je englesko ime. Na engleskom jeziku značenje imena Nigel je: Šampion. Potiče od irskog i škotskog imena Niall. Skandinavsko značenje: Ime Nigel je skandinavsko'
 'Etimologija i istorijsko poreklo - Nigel. Nigel je englesko muško ime koje potiče od "Nigellus", srednjovekovne latinske forme imena Neil. Neil je zapravo anglizirana forma dugotrajnog galskog imena Niall (nijal "izgovara se") neel koje se koristi još od Srednjeg veka u Irskoj i škotskim visoravnima'
 'Značenje imena Nigel. Englesko ime. Na engleskom jeziku, ime Nigel znači - Šampion. Potiče od irskog i škotskog imena Niall. Ostali poreklo za ime Nigel uključuje - Englesko, Američko, Galsko, Irska. 

In [162]:
loaded_table = pq.read_table('datasets/naquestions.parquet')
df_natural_language = loaded_table.to_pandas()

In [165]:
df_natural_language

,id,query,passage_text
0,-7198839892825072919,Ko je bio beba u filmu Duhovi 2?,[Vilijam T. Dojčendorf i Henk J. Dojčendorf II...
1,2020682570490858816,Ko peva u filmu Cry Baby za Džonija Depa?,[Džejms Intveld kao glas za pevanje lika Cry-B...
2,2936806557797920433,64-bit verzija Microsoft Windows-a ne podržava...,[Podrška za virtualni 8086 režim nije dostupna...
3,2057977686951937626,Ko je igrao Dejvida u ubistvu Đanija Versačea?,[Kodi Fern kao Dejvid Medson]
4,1318031841813121387,ko je pevao temu pesmu za seriju 70-ih godina,"[Početna tema ""In the Street"" od grupe Big Sta..."
...,...,...,...
86,-3576303950778028254,fc barcelona vs real madrid last 10 matches,[# Datum R . Domaći tim Gostujući tim Rezultat...
87,4245798066923223457,ko ima najviše nagrada za najkorisnijeg igrača...,[Bob Pettit i Kobe Bryant su jedini igrači koj...
88,-7155063525359960751,koliko nfl utakmica je odigrano u Londonu,[Godina Datum Određeni gost Rezultat Određeni ...
89,8932885154379264842,who rides so late through night and wind,[Ko tako kasno jaše kroz noć i vetar? To je ot...


In [167]:
row_89 = df_natural_language.iloc[89]  # Indexing is 0-based, so 89th row is at index 88
print(row_89)

id                                            8932885154379264842
query                    who rides so late through night and wind
passage_text    [Ko tako kasno jaše kroz noć i vetar? To je ot...
Name: 89, dtype: object


In [171]:
for q in df_natural_language['passage_text']:
    print(q)

['Vilijam T. Dojčendorf i Henk J. Dojčendorf II kao beba Oskar']
['Džejms Intveld kao glas za pevanje lika Cry-Baby-ja.']
['Podrška za virtualni 8086 režim nije dostupna u x86-64 long režimu, iako je i dalje prisutna na procesorima sposobnim za 64-bitni rad u legacy režimu.']
['Kodi Fern kao Dejvid Medson']
['Početna tema "In the Street" od grupe Big Star izvođena od strane: Tod Grifin (sezona 1) i Cheap Trick (sezone 2 - 8)']
['Depozicija je termodinamički proces, faza prelaza u kojoj gas prelazi u čvrsto stanje bez prolaska kroz tečnu fazu. Suprotnost depozicije je sublimacija i zbog toga se ponekad depozicija naziva desublimacija.']
["`` Only Love Can Break Your Heart '' je pesma koju je napisao Nil Jang. Izvodili su je mnogi bendovi, uključujući singl iz 1990. godine od strane benda Saint Etienne."]
['Međunarodna destinacija za ovaj ciklus bila je Bali, Indonezija. Pobednica je bila 19-godišnja Jourdan Miller iz Bend, Oregon.']
['Senat Sjedinjenih Američkih Država se sastoji od 100

In [91]:
type(df['passage_text'][0])

numpy.ndarray

## Testing GPT Response

In [95]:
response_path = Path("./translation_pipeline_test/naquestions_test_results.jsonl")
assert response_path.exists(), f"File not found: {response_path}"

In [109]:
responses = []
with open(response_path, "r", encoding="utf-8") as file:
    for i, line in enumerate(file):
        data = json.loads(line)
        request = data[0]
        response = data[1]['choices'][0]['message']['content']
        try:
            response_json = json.loads(response)
            responses.append(response_json)
        except json.JSONDecodeError as e:
            error_data = response
            print(f"Index: {i}, Error: {e}")

Index: 7, Error: Expecting ',' delimiter: line 5 column 1461 (char 1583)


In [111]:
print(error_data)

{
    "translations": [
        {
            "query": "ko ima najviše nagrada za najkorisnijeg igrača All-Star utakmica",
            "passage_text": ["Bob Pettit i Kobe Bryant su jedini igrači koji su osvojili nagradu za najkorisnijeg igrača All-Star utakmica četiri puta. Oscar Robertson, Majkl Džordan, Šekil O'Nil i Lebron Džejms su svaki osvojili nagradu tri puta, dok su Bob Cousy, Džulijus Erving, Azejja Tomas, Međik Džonson, Karl Meloun, Alen Ajverson i Rasel Vestbruk svi osvojili nagradu dva puta. Džejmsova prva nagrada za najkorisnijeg igrača All-Star utakmica 2006. godine učinila ga je najmlađim ikada koji je osvojio nagradu sa 21 godinom i 1 mesecom. Kajri Irving, pobednik nagrade za najkorisnijeg igrača All-Star utakmica 2014. godine, drugi je najmlađi sa 21 godinom i 10 meseci. Oni su značajni jer su najmlađi koji su osvojili nagradu, obojica kao igrači Klivlenda. Četiri puta su postojali zajednički pobednici - Elgin Bejlor i Pettit 1959. godine, Džon Stokton i Meloun 1993.

In [108]:
[{"model": "gpt-3.5-turbo-0125", "response_format": {"type": "json_object"}, "temperature": 0, "messages": [{"role": "system", "content": "\n***TRANSLATION FROM ENGLISH TO SERBIAN***\n\n**GOALS**\n\nYou are a professional translator fluent in English and Serbian. \nYour primary goal is to produce a high-quality, natural-sounding translation from English to Serbian. \nYou are translating texts and questions pertaining to the texts. The translation is intended for dataset creation. Look at the example below:\n\n***TRANSLATION EXAMPLE***\n***ENGLISH***\nquery: 'What is a unicorn?'\npassage_text: 'The unicorn is a legendary creature that has been described since antiquity as a beast with a single large, pointed, spiraling horn projecting from its forehead.'\n\n***SERBIAN TRANSLATION***\nquery: 'Šta je jednorog?'\npassage_text: 'Jednorog je mitsko stvorenje koje se od davnina opisuje kao zver sa jednim velikim, šiljastim, spiralnim rogom koji mu viri iz čela.'\n***END OF TRANSLATION EXAMPLE***\n\nTo translate, follow the steps below:\n   **TRANSLATION INSTRUCTIONS**\n   1. Read and understand the sentence in English.\n   2. When you understand the English sentence, start to translate.\n   3. Pay close attention to both left and right context when you are making translation decisions.  4. Convey the original context, tone and meaning in the Serbian translation.\n   4. Avoid literal translations and ensure the output reads naturally in Serbian.\n   5. The translation must be contextually accurate, fluent, and adhere to the grammatical rules and lexicon of the Serbian language.\n   6. The declination of nouns, adjectives, and pronouns must be correct.\n   7. Make sure to proofread the translated text in Serbian and revise any mistakes. If no revisions are needed, provide the translations as they are.\n\n   **FORMATTING INSTRUCTIONS**\n   1. Strings should be enclosed within single quotation marks ('').\n   2. Use double quotes for strings and escape internal quotes with a backslashes (\\).\n \n   **OUTPUT FORMATTING**\n   - Ensure the output is a valid JSON file, parsable by Python's json.loads().\n   - Ensure consistent JSON formatting as illustrated in the example below:\n\n      **EXAMPLE**\n      {\"translations\" : [{\"query\" : 'This is a query',\n                         \"passage_text\" : ['This is one passage. With another sentence.',\n                                          'This is yet another passage. With yet another sentence.']\n                        }\n                        ]\n      }\n       **END OF EXAMPLE**\n\n   - Strictly follow the structure provided in the example when generating the output.   \n   - Make sure to translate text under both \"query\" and \"passage_text\" keys.\n\n"}, {"role": "user", "content": "{\"query_id\": \"4245798066923223457\", \"query\": \"who has the most all star mvp awards\", \"passage_text\": [\"Bob Pettit and Kobe Bryant are the only two players to win the All - Star Game MVP four times . Oscar Robertson , Michael Jordan , Shaquille O'Neal , and LeBron James have each won the award three times , while Bob Cousy , Julius Erving , Isiah Thomas , Magic Johnson , Karl Malone , Allen Iverson , and Russell Westbrook have all won the award twice . James ' first All - Star MVP in 2006 made him the youngest to have ever won the award at the age of 21 years , 1 month . Kyrie Irving , winner of the 2014 All - Star Game MVP , is the second - youngest at 21 years , 10 months . They are notable as being the two youngest to win the award , both as Cleveland Cavaliers . Four of the games had joint winners -- Elgin Baylor and Pettit in 1959 , John Stockton and Malone in 1993 , O'Neal and Tim Duncan in 2000 , and O'Neal and Bryant in 2009 . O'Neal became the first player in All - Star history to share two MVP awards as well as the first player to win the award with multiple teams . The Los Angeles Lakers have had eleven winners while the Boston Celtics have had eight . Duncan of the U.S. Virgin Islands and Irving of Australia are the only winners not born in the United States . Both Duncan and Irving are American citizens , but are considered `` international '' players by the NBA because they were not born in one of the fifty states or Washington , D.C. No player trained entirely outside the U.S. has won the award ; Irving lived in the U.S. since age two , and Duncan played U.S. college basketball at Wake Forest .\"]}"}]}, {"id": "chatcmpl-9yelcepUFNOTyrdTQR677DyHkQ49b", "object": "chat.completion", "created": 1724243416, "model": "gpt-3.5-turbo-0125", "choices": [{"index": 0, "message": {"role": "assistant", "content": "{\n    \"translations\": [\n        {\n            \"query\": \"ko ima najviše nagrada za najkorisnijeg igrača All-Star utakmica\",\n            \"passage_text\": [\"Bob Pettit i Kobe Bryant su jedini igrači koji su osvojili nagradu za najkorisnijeg igrača All-Star utakmica četiri puta. Oscar Robertson, Majkl Džordan, Šekil O'Nil i Lebron Džejms su svaki osvojili nagradu tri puta, dok su Bob Cousy, Džulijus Erving, Azejja Tomas, Međik Džonson, Karl Meloun, Alen Ajverson i Rasel Vestbruk svi osvojili nagradu dva puta. Džejmsova prva nagrada za najkorisnijeg igrača All-Star utakmica 2006. godine učinila ga je najmlađim ikada koji je osvojio nagradu sa 21 godinom i 1 mesecom. Kajri Irving, pobednik nagrade za najkorisnijeg igrača All-Star utakmica 2014. godine, drugi je najmlađi sa 21 godinom i 10 meseci. Oni su značajni jer su najmlađi koji su osvojili nagradu, obojica kao igrači Klivlenda. Četiri puta su postojali zajednički pobednici - Elgin Bejlor i Pettit 1959. godine, Džon Stokton i Meloun 1993. godine, O'Nil i Tim Dankan 2000. godine, i O'Nil i Brajant 2009. godine. O'Nil je postao prvi igrač u istoriji All-Star utakmica koji je podelio dve MVP nagrade, kao i prvi igrač koji je osvojio nagradu sa više timova. Los Anđeles Lejkersi imaju jedanaest pobednika dok Boston Seltiksi imaju osam. Dankan sa Devičanskih Ostrva i Irving iz Australije su jedini pobednici koji nisu rođeni u Sjedinjenim Američkim Državama. I Dankan i Irving su američki državljani, ali ih NBA smatra \"                                                                                                    ", "refusal": null}, "logprobs": null, "finish_reason": "stop"}], "usage": {"prompt_tokens": 967, "completion_tokens": 667, "total_tokens": 1634}, "system_fingerprint": null}, {"id": "4245798066923223457"}]


NameError: name 'null' is not defined

In [124]:
data_m = load_dataset("microsoft/ms_marco", "v1.1", cache_dir=dir)


In [128]:
data_m['test']['query_id']

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [133]:
lala = "Player No . Team ( s ) Championships Notes Shefflin , Henry Henry Shefflin 10 Kilkenny 2000 , 2002 , 2003 , 2006 , 2007 , 2008 , 2009 , 2011 , 2012 , 2014 Highest number of medals for a single player Delaney , J.J. J.J. Delaney 9 Kilkenny 2002 , 2003 , 2006 , 2007 , 2008 , 2009 , 2011 , 2012 , 2014 Won a medal as a non-playing substitute in 2006 Tyrrell , Jackie Jackie Tyrrell 9 Kilkenny 2003 , 2006 , 2007 , 2008 , 2009 , 2011 , 2012 , 2014 , 2015 Won a medal as a non-playing substitute in 2003 and 2015 Hickey , Noel Noel Hickey 9 Kilkenny 2000 , 2002 , 2003 , 2006 , 2007 , 2008 , 2009 , 2011 , 2012 Won a medal as a non-playing substitute in 2009 Walsh , Tommy Tommy Walsh 9 Kilkenny 2002 , 2003 , 2006 , 2007 , 2008 , 2009 , 2011 , 2012 , 2014 Won medals as a non-playing substitute in 2002 and 2014 Skehan , Noel Noel Skehan 9 Kilkenny 1963 , 1967 , 1969 , 1972 , 1974 , 1975 , 1979 , 1982 , 1983 Won medals as a non-playing substitute in 1963 , 1967 and 1969 Power , Richie Richie Power 8 Kilkenny 2006 , 2007 , 2008 , 2009 , 2011 , 2012 , 2014 , 2015 Larkin , Eoin Eoin Larkin 8 Kilkenny 2006 , 2007 , 2008 , 2009 , 2011 , 2012 , 2014 , 2015 Brennan , Eddie Eddie Brennan 8 Kilkenny 2000 , 2002 , 2003 , 2006 , 2007 , 2008 , 2009 , 2011 Cummins , Frank Frank Cummins 8 Kilkenny 1967 , 1969 , 1972 , 1974 , 1975 , 1979 , 1982 , 1983 Won a medal as a non-playing substitute in 1967 Kavanagh , Michael Michael Kavanagh 8 Kilkenny 2000 , 2002 , 2003 , 2006 , 2007 , 2008 , 2009 , 2011 Won a medal as a non-playing substitute in 2011 Fennelly , Michael Michael Fennelly 8 Kilkenny 2006 , 2007 , 2008 , 2009 , 2011 , 2012 , 2014 , 2015 Won medals as a non-playing substitute in 2006 and 2008 Fogarty , Aidan Aidan Fogarty 8 Kilkenny 2003 , 2006 , 2007 , 2008 , 2009 , 2011 , 2012 , 2014 Won medals as a non-playing substitute in 2003 , 2011 & 2014 Doyle , John John Doyle 8 Tipperary 1949 , 1950 , 1951 , 1958 , 1961 , 1962 , 1964 , 1965 Winner of medals in three separate decades Ring , Christy Christy Ring 8 Cork 1941 , 1942 , 1943 , 1944 , 1946 , 1952 , 1953 , 1954 Hogan , Brian Brian Hogan 7 Kilkenny 2006 , 2007 , 2008 , 2009 , 2011 , 2012 , 2014 Won a medal as a non-playing substitute in 2006 and 2014 Doyle , Dick Dick Doyle 7 Kilkenny 1904 , 1905 , 1907 , 1909 , 1911 , 1912 , 1913 Rochford , Jack Jack Rochford 7 Kilkenny 1904 , 1905 , 1907 , 1909 , 1911 , 1912 , 1913 Walsh , Dick Dick Walsh 7 Kilkenny 1904 , 1905 , 1907 , 1909 , 1911 , 1912 , 1913 Walton , Sim Sim Walton 7 Kilkenny 1904 , 1905 , 1907 , 1909 , 1911 , 1912 , 1913 Reid , T.J. T.J. Reid 7 Kilkenny 2007 , 2008 , 2009 , 2011 , 2012 , 2014 , 2015 Won an All - Ireland medal as a non-playing substitute in 2007 Hogan , Richie Richie Hogan 7 Kilkenny 2007 , 2008 , 2009 , 2011 , 2012 , 2014 , 2015 Won an All - Ireland medal as a non-playing substitute in 2007 and 2008 Comerford , Martin Martin Comerford 6 Kilkenny 2002 , 2003 , 2006 , 2007 , 2008 , 2009 Doyle , Ned Ned Doyle 6 Kilkenny 1904 , 1905 , 1907 , 1909 , 1911 , 1912 Keher , Eddie Eddie Keher 6 Kilkenny 1963 , 1967 , 1969 , 1972 , 1974 , 1975 Kelly , Jimmy Jimmy Kelly 6 Kilkenny 1905 , 1907 , 1909 , 1911 , 1912 , 1913 Kennedy , Dan Dan Kennedy 6 Kilkenny 1905 , 1907 , 1909 , 1911 , 1912 , 1913 Lanigan , Paddy Paddy Lanigan 6 Kilkenny 1904 , 1905 , 1907 , 1909 , 1911 , 1912 Lyng , Derek Derek Lyng 6 Kilkenny 2002 , 2003 , 2006 , 2007 , 2008 , 2009 McGarry , James James McGarry 6 Kilkenny 2000 , 2002 , 2003 , 2006 , 2007 , 2008 Won a medal as a non-playing substitute in 2007 Doyle , Jimmy Jimmy Doyle 6 Tipperary 1958 , 1961 , 1962 , 1964 , 1965 , 1971 Winner of medals in three separate decades Carey , D.J. D.J. Carey 5 Kilkenny 1992 , 1993 , 2000 , 2002 , 2003 Doherty , Dick Dick Doherty 5 Kilkenny 1907 , 1909 , 1911 , 1912 , 1913 Doyle , Mick Mick Doyle 5 Kilkenny 1907 , 1909 , 1911 , 1912 , 1913 Fitzpatrick , Billy Billy Fitzpatrick 5 Kilkenny 1974 , 1975 , 1979 , 1982 , 1983 Gargan , Matt Matt Gargan 5 Kilkenny 1907 , 1909 , 1911 , 1912 , 1913 Grace , Dick Dick Grace 5 Kilkenny 1909 , 1911 , 1912 , 1913 , 1922 Winner of medals in three separate decades Henderson , Pat Pat Henderson 5 Kilkenny 1967 , 1969 , 1972 , 1974 , 1975 Howard , Garrett Garrett Howard 5 Limerick Dublin 1921 , 1924 , 1927 , 1934 , 1936 Keoghan , Jack Jack Keoghan 5 Kilkenny 1907 , 1909 , 1911 , 1912 , 1913 Larkin , Phil Phil Larkin 5 Kilkenny 1963 , 1972 , 1974 , 1975 , 1979 Fitzpatrick , James James Fitzpatrick 5 Kilkenny 2006 , 2007 , 2008 , 2009 , 2011 Won medals as a non-playing substitute in 2009 and 2011 Lotty , Alan Alan Lotty 5 Cork 1941 , 1942 , 1943 , 1944 , 1946 Lynch , Jack Jack Lynch 5 Cork 1941 , 1942 , 1943 , 1944 , 1946 Won a football medal in 1945 Maher , Michael Michael Maher 5 Tipperary 1958 , 1961 , 1962 , 1964 , 1965 Maher , Mikey Mikey Maher 5 Tipperary 1895 , 1896 , 1898 , 1899 , 1900 McCarthy , Charlie Charlie McCarthy 5 Cork 1966 , 1970 , 1976 , 1977 , 1978 McCarthy , Gerald Gerald McCarthy 5 Cork 1966 , 1970 , 1976 , 1977 , 1978 Murphy , Willie Willie Murphy 5 Cork 1941 , 1942 , 1943 , 1944 , 1946 Nealon , Donie Donie Nealon 5 Tipperary 1958 , 1961 , 1962 , 1964 , 1965 O\'Donovan , Paddy Paddy O\'Donovan 5 Cork 1941 , 1942 , 1943 , 1944 , 1946 Ahern , Paddy Paddy Ahern 5 Cork 1919 , 1926 , 1928 , 1929 , 1931 Winner of medals in three separate decades Barry - Murphy , Jimmy Jimmy Barry - Murphy 5 Cork 1976 , 1977 , 1978 , 1984 , 1986 Won a football medal in 1973 Buckley , Din Joe Din Joe Buckley 5 Cork 1941 , 1942 , 1943 , 1944 , 1946 Byrne , Mickey Mickey Byrne 5 Tipperary 1945 , 1949 , 1950 , 1951 , 1958 Carey , Kieran Kieran Carey 5 Tipperary 1958 , 1961 , 1962 , 1964 , 1965 Crowley , Johnny Johnny Crowley 5 Cork 1976 , 1977 , 1978 , 1984 , 1986 Devaney , Liam Liam Devaney 5 Tipperary 1958 , 1961 , 1962 , 1964 , 1965 Power , Matty Matty Power 5 Kilkenny Dublin 1922 , 1927 , 1932 , 1933 , 1935 Wall , Tony Tony Wall 5 Tipperary 1958 , 1961 , 1962 , 1964 , 1965 Walsh , Johnny Johnny Walsh 5 Tipperary 1895 , 1896 , 1898 , 1899 , 1900 Doyle , Tommy Tommy Doyle 5 Tipperary 1937 , 1945 , 1949 , 1950 , 1951 Winner of medals in three separate decades English , Theo Theo English 5 Tipperary 1958 , 1961 , 1962 , 1964 , 1965 Young , Jim Jim Young 5 Cork 1941 , 1942 , 1943 , 1944 , 1946 Coughlan , Eudie Eudie Coughlan 5 Cork 1919 , 1926 , 1928 , 1929 , 1931 Won a medal as a non-playing substitute in 1919 Ahern , Mick Mick Ahern Cork 1926 , 1928 , 1929 , 1931 Barry - Murphy , Dinny Dinny Barry - Murphy Cork 1926 , 1928 , 1929 , 1931 Brennan , J.J. J.J. Brennan Kilkenny 1905 , 1911 , 1912 , 1913 Brennan , Tony Tony Brennan Tipperary 1945 , 1949 , 1950 , 1951 Burns , Mick Mick Burns Tipperary 1961 , 1962 , 1964 , 1965 Byrne , Phil Phil Byrne Tipperary 1895 , 1896 , 1898 , 1899 Cashman , Tom Tom Cashman Cork 1977 , 1978 , 1984 , 1986 Coogan , Martin Martin Coogan Kilkenny 1963 , 1967 , 1969 , 1972 Cottrell , Con Con Cottrell Cork 1941 , 1943 , 1944 , 1946 Coughlan , Denis Denis Coughlan Cork 1970 , 1976 , 1977 , 1978 Won a football medal in 1973 Crotty , Mick Mick Crotty Kilkenny 1972 , 1974 , 1975 , 1979 Cummins , Ray Ray Cummins Cork 1970 , 1976 , 1977 , 1978 Won a football medal in 1973 Daly , Tommy Tommy Daly Dublin Clare 1917 , 1920 , 1924 , 1927 Delaney , Pat Pat Delaney Kilkenny 1969 , 1972 , 1974 , 1975 Delea , Paddy Paddy Delea Cork 1926 , 1928 , 1929 , 1931 Fennelly , Colin Colin Fennelly Kilkenny 2011 , 2012 , 2014 , 2015 Fogarty , Conor Conor Fogarty Kilkenny 2011 , 2012 , 2014 , 2015 Heffernan , Christy Christy Heffernan Kilkenny 1982 , 1983 , 1992 , 1993 Horgan , John John Horgan Cork 1970 , 1976 , 1977 , 1978 Hurley , Jim Jim Hurley Cork 1926 , 1928 , 1929 , 1931 Joyce , Kieran Kieran Joyce Kilkenny 2011 , 2012 , 2014 , 2015 Won a medal as a non-playing substitute in 2011 Larkin , Paddy Paddy Larkin Kilkenny 1932 , 1933 , 1935 , 1939 Maher , Ed Ed Maher Tipperary 1895 , 1896 , 1898 , 1900 Maher , Jack Jack Maher Tipperary 1895 , 1896 , 1898 , 1899 McKenna , John John McKenna Tipperary 1961 , 1962 , 1964 , 1965 McLoughlin , Se\\u00e1n Se\\u00e1n McLoughlin Tipperary 1961 , 1962 , 1964 , 1965 Moran , Paddy Paddy Moran Kilkenny 1963 , 1967 , 1969 , 1972 Murphy , Con Con Murphy Cork 1942 , 1943 , 1944 , 1946 Murphy , Eoin Eoin Murphy Kilkenny 2011 , 2012 , 2014 , 2015 Won a medal as a non-playing substitute in 2011 & 2012 Murphy , Paul Paul Murphy Kilkenny 2011 , 2012 , 2014 , 2015 O\'Brien , Liam Liam O\'Brien Kilkenny 1972 , 1974 , 1975 , 1979 O\'Connell , Edward Edward O\'Connell Cork 1926 , 1928 , 1929 , 1931 O\'Leary , Se\\u00e1nie Se\\u00e1nie O\'Leary Cork 1976 , 1977 , 1978 , 1984 O\'Regan , Jim Jim O\'Regan Cork Dublin 1926 , 1928 , 1929 , 1931 O\'Riordan , Gerry Gerry O\'Riordan Cork 1946 , 1952 , 1953 , 1954 Phelan , Paddy Paddy Phelan Kilkenny 1932 , 1933 , 1935 , 1939 Power , John John Power Kilkenny 1992 , 1993 , 2000 , 2002 Power , John T. John T. Power Kilkenny 1907 , 1911 , 1912 , 1913 Quirke , Johnny Johnny Quirke Cork 1941 , 1942 , 1943 , 1944 Ryan , Lester Lester Ryan Kilkenny 2011 , 2012 , 2014 , 2015 Won medals as a non-playing substitute in 2011 , 2012 & 2015 Ryan , Tommy Tommy Ryan Tipperary 1896 , 1898 , 1899 , 1900 Thornhill , Batt Batt Thornhill Cork 1941 , 1942 , 1943 , 1944 Treacy , Jim Jim Treacy Kilkenny 1967 , 1969 , 1972 , 1974 Walsh , Jimmy Jimmy Walsh Kilkenny 1932 , 1933 , 1935 , 1939 Walsh , Ollie Ollie Walsh Kilkenny 1957 , 1963 , 1967 , 1969 Anthony , Jack Jack Anthony Kilkenny 1904 , 1905 , 1907 Bannon , S\\u00e9amus S\\u00e9amus Bannon Tipperary 1949 , 1950 , 1951 Barry , Paddy Paddy Barry Cork 1952 , 1953 , 1954 Barry , Peter Peter Barry Kilkenny 2000 , 2002 , 2003 Barry , Tom Tom Barry Cork 1928 , 1929 , 1931 Brennan , Mick Mick Brennan Kilkenny 1974 , 1975 , 1979 Brennan , Ned Ned Brennan Tipperary 1895 , 1896 , 1898 Buckley , Cillian Cillian Buckley Kilkenny 2012 , 2014 , 2015 Byrne , Eddie Eddie Byrne Kilkenny Dublin 1932 , 1933 , 1935 Byrne , Podge Podge Byrne Kilkenny 1932 , 1933 , 1935 Carroll , Ted Ted Carroll Kilkenny 1963 , 1967 , 1969 Cashman , John John Cashman Cork 1892 , 1893 , 1894 Clohessy , Paddy Paddy Clohessy Limerick 1934 , 1936 , 1940 Cody , Brian Brian Cody Kilkenny 1975 , 1982 , 1983 Coleman , Martin Martin Coleman Cork 1976 , 1977 , 1978 Comerford , Andy Andy Comerford Kilkenny 2000 , 2002 , 2003 Condon , Se\\u00e1n Se\\u00e1n Condon Cork 1942 , 1943 , 1944 Condon , Tim Tim Condon Tipperary 1896 , 1898 , 1899 Connolly , Johnny Johnny Connolly Tipperary 1895 , 1896 , 1898 Corcoran , Brian Brian Corcoran Cork 1999 , 2004 , 2005 Creedon , Dave Dave Creedon Cork 1952 , 1953 , 1954 Crotty , Mick Mick Crotty Kilkenny 1974 , 1975 , 1979 Crowley , Tim Tim Crowley Cork 1977 , 1978 , 1984 Cunningham , Ger Ger Cunningham Cork 1984 , 1986 , 1990 Cusack , Donal \\u00d3g Donal \\u00d3g Cusack Cork 1999 , 2004 , 2005 Daly , Willie John Willie John Daly Cork 1952 , 1953 , 1954 Deane , Joe Joe Deane Cork 1999 , 2004 , 2005 Devane , Will Will Devane Tipperary 1895 , 1896 , 1898 Dillon , Pa Pa Dillon Kilkenny 1967 , 1969 , 1972 Doherty , Bob Bob Doherty Clare Dublin 1914 , 1920 , 1924 Dooley , Joe Joe Dooley Offaly 1985 , 1994 , 1998 English , Jim Jim English Wexford 1955 , 1956 , 1960 Fennelly , Ger Ger Fennelly Kilkenny 1979 , 1982 , 1983 Fennelly , Liam Liam Fennelly Kilkenny 1982 , 1983 , 1992 Finn , Jimmy Jimmy Finn Tipperary 1950 , 1951 , 1958 Flood , Tim Tim Flood Wexford 1955 , 1956 , 1960 Fouhy , Matty Matty Fouhy Cork 1952 , 1953 , 1954 Gill , Mick Mick Gill Galway Dublin 1923 , 1924 , 1927 Grace , Pierce Pierce Grace Kilkenny 1911 , 1912 , 1913 Won football medals with Dublin in 1906 and 1907 Hayes , Conor Conor Hayes Galway 1980 , 1987 , 1988 Hayes , Martin Martin Hayes Dublin 1917 , 1920 , 1927 Henderson , Ger Ger Henderson Kilkenny 1979 , 1982 , 1983 Henderson , John John Henderson Kilkenny 1979 , 1982 , 1983 Hennessy , Joe Joe Hennessy Kilkenny 1979 , 1982 , 1983 Hennessy , Kevin Kevin Hennessy Cork 1984 , 1986 , 1990 Hough , John John Hough Tipperary 1951 , 1958 , 1961 Hoyne , John John Hoyne Kilkenny 2000 , 2002 , 2003 Kehoe , Padge Padge Kehoe Wexford 1955 , 1956 , 1960 Kennedy , Mick Mick Kennedy Limerick 1934 , 1936 , 1940 Kenny , Paddy Paddy Kenny Tipperary 1949 , 1950 , 1951 Kenny , Se\\u00e1n Se\\u00e1n Kenny Tipperary 1949 , 1950 , 1951 Lane , Noel Noel Lane Galway 1980 , 1987 , 1988 Lawlor , Pat Pat Lawlor Kilkenny 1972 , 1974 , 1975 Leahy , Tommy Tommy Leahy Kilkenny 1932 , 1933 , 1935 Linnane , Sylvie Sylvie Linnane Galway 1980 , 1987 , 1988 Lyons , John John Lyons Cork 1952 , 1953 , 1954 Mackey , John John Mackey Limerick 1934 , 1936 , 1940 Mackey , Mick Mick Mackey Limerick 1934 , 1936 , 1940 Madden , Morgan Morgan Madden Cork 1928 , 1929 , 1931 Maher , John John Maher Tipperary 1930 , 1937 , 1945 Maher , Sonny Sonny Maher Tipperary 1949 , 1950 , 1951 McCarthy , Timmy Timmy McCarthy Cork 1999 , 2004 , 2005 McCurtain , Dermot Dermot McCurtain Cork 1977 , 1978 , 1984 Meagher , Lory Lory Meagher Kilkenny 1932 , 1933 , 1935 Mockler , Bob Bob Mockler Tipperary Dublin 1917 , 1920 , 1924 Also won a medal as a non-playing substitute in 1908 Morrissey , Jim Jim Morrissey Wexford 1955 , 1956 , 1960 Morrissey , Mick Mick Morrissey Wexford 1955 , 1956 , 1960 Moylan , Pat Pat Moylan Cork 1976 , 1977 , 1978 Mulcahy , Tom Tom Mulcahy Cork 1943 , 1944 , 1946 Mulcahy , Tom\\u00e1s Tom\\u00e1s Mulcahy Cork 1984 , 1986 , 1990 Mullally , Richie Richie Mullally Kilkenny 2002 , 2003 , 2006 Murphy , Brian Brian Murphy Cork 1976 , 1977 , 1978 Won a football medal in 1973 Murphy , Gerald Gerald Murphy Cork 1952 , 1953 , 1954 Murphy , Se\\u00e1n \\u00d3g Se\\u00e1n \\u00d3g Murphy Cork 1919 , 1926 , 1928 \\u00d3 hAilp\\u00edn , Se\\u00e1n \\u00d3g Se\\u00e1n \\u00d3g \\u00d3 hAilp\\u00edn Cork 1999 , 2004 , 2005 O\'Connell , Jimmy Jimmy O\'Connell Kilkenny 1933 , 1935 , 1939 O\'Connell , Mick Mick O\'Connell Cork 1928 , 1929 , 1931 O\'Connor , Ben Ben O\'Connor Cork 1999 , 2004 , 2005 O\'Connor , John John O\'Connor Cork 1892 , 1893 , 1894 O\'Connor , Willie Willie O\'Connor Kilkenny 1992 , 1993 , 2000 O\'Doherty , Martin Martin O\'Doherty Cork 1976 , 1977 , 1978 O\'Donnell , Nick Nick O\'Donnell Wexford 1955 , 1956 , 1960 O\'Grady , Peter Peter O\'Grady Cork 1928 , 1929 , 1931 O\'Hara , Dick Dick O\'Hara Kilkenny 1979 , 1982 , 1983 O\'Keeffe , Joe Joe O\'Keeffe Tipperary 1898 , 1899 , 1900 O\'Reilly , Peter Peter O\'Reilly Kilkenny 1932 , 1933 , 1935 O\'Shaughnessy , Tony Tony O\'Shaughnessy Cork 1952 , 1953 , 1954 O\'Sullivan , Diarmuid Diarmuid O\'Sullivan Cork 1999 , 2004 , 2005 O\'Sullivan , Tony Tony O\'Sullivan Cork 1984 , 1986 , 1990 Phelan , Joe Joe Phelan Laois Dublin 1915 , 1917 , 1920 Prendergast , Paddy Paddy Prendergast Kilkenny 1979 , 1982 , 1983 Purcell , Kieran Kieran Purcell Kilkenny 1972 , 1974 , 1975 Rackard , Billy Billy Rackard Wexford 1955 , 1956 , 1960 Reddin , Tony Tony Reddin Tipperary 1949 , 1950 , 1951 Roche , Jim Jim Roche Limerick 1934 , 1936 , 1940 Roche , Mick Mick Roche Tipperary 1964 , 1965 , 1971 Ryan , Declan Declan Ryan Tipperary 1989 , 1991 , 2001 Ryan , Mick Mick Ryan Tipperary 1949 , 1950 , 1951 Ryan , P.J. P.J. Ryan Kilkenny 2007 , 2008 , 2009 Ryan , Timmy Timmy Ryan Limerick 1934 , 1936 , 1940 Semple , Tom Tom Semple Tipperary 1900 , 1906 , 1908 Shanahan , Phil Phil Shanahan Tipperary 1949 , 1950 , 1951 Shelly , Hugh Hugh Shelly Tipperary 1906 , 1908 , 1916 Stakelum , Pat Pat Stakelum Tipperary 1949 , 1950 , 1951 Stapleton , D.J. D.J. Stapleton Kilkenny 1904 , 1905 , 1907 Tennyson , John John Tennyson Kilkenny 2006 , 2007 , 2009 Twomey , Vincy Vincy Twomey Cork 1952 , 1953 , 1954 Wall , Mike Mike Wall Tipperary 1896 , 1899 , 1900 Walsh , Jim Jim Walsh Dublin 1920 , 1924 , 1927 Walsh , Walter Walter Walsh Kilkenny 2012 , 2014 , 2015 Wheeler , Ned Ned Wheeler Wexford 1955 , 1956 , 1960 White , Martin Martin White Kilkenny 1932 , 1933 , 1935 Aylward , Pat Pat Aylward Kilkenny Dublin 1922 , 1924 Allen , Tom Tom Allen Tipperary 1900 , 1906 Baker , Ollie Ollie Baker Clare 1995 , 1997 Bannon , Joe Joe Bannon Dublin 1924 , 1927 Barry , Paddy Paddy Barry Cork 1966 , 1970 Barry , Tom Tom Barry Dublin 1924 , 1927 Bermingham , Brendan Brendan Bermingham Offaly 1981 , 1985 Blanchfield , Peter Peter Blanchfield Kilkenny 1935 , 1939 Bonnar , Colm Colm Bonnar Tipperary 1989 , 1991 Bonnar , Conal Conal Bonnar Tipperary 1989 , 1991 Bonnar , Cormac Cormac Bonnar Tipperary 1989 , 1991 Brennan , Jamesie Jamesie Brennan Kilkenny 1992 , 1993 Brennan , Kieran Kieran Brennan Kilkenny 1982 , 1983 Brennan , Micka Micka Brennan Cork 1941 , 1943 Brennan , Nickey Nickey Brennan Kilkenny 1979 , 1982 Brolan , Paddy Paddy Brolan Tipperary 1906 , 1908 Browne , John John Browne Cork 1999 , 2004 Buckley , Pat Pat Buckley Cork 1890 , 1892 Burke , Frank Frank Burke Dublin 1917 , 1920 Burke , Jimmy Jimmy Burke Tipperary 1906 , 1908 Burke , Miah Miah Burke Cork 1928 , 1929 Burke , Paddy Paddy Burke Tipperary 1906 , 1908 Butler , Tom Tom Butler Tipperary 1930 , 1937 Buttimer , Jim Jim Buttimer Cork 1941 , 1942 Cahill , Phil Phil Cahill Tipperary 1925 , 1930 Callanan , John Joe John Joe Callanan Dublin Tipperary 1920 , 1930 Carr , Declan Declan Carr Tipperary 1989 , 1991 Carter , Charlie Charlie Carter Kilkenny 2000 , 2002 Cashman , Jim Jim Cashman Cork 1986 , 1990 Clancy , Conor Conor Clancy Clare 1995 , 1997 Cleary , James James Cleary Dublin 1917 , 1920 Cleary , Michael Michael Cleary Tipperary 1989 , 1991 Cleere , S\\u00e9amus S\\u00e9amus Cleere Kilkenny 1963 , 1967 Clohessy , Dave Dave Clohessy Limerick 1934 , 1936 Clohessy , Se\\u00e1n Se\\u00e1n Clohessy Kilkenny 1957 , 1963 Coffey , Flor Flor Coffey Tipperary 1945 , 1949 Collins , Paddy Paddy Collins Cork 1929 , 1931 Commins , John John Commins Galway 1987 , 1988 Connolly , Liam Liam Connolly Tipperary 1958 , 1962 Considine , Brendan Brendan Considine Clare Dublin 1914 , 1917 Coogan , Jimmy Jimmy Coogan Kilkenny 2002 , 2003 Cooney , Joe Joe Cooney Galway 1987 , 1988 Corbett , Lar Lar Corbett Tipperary 2001 , 2010 Cornally , Ger Ger Cornally Tipperary 1937 , 1945 Corrigan , Mark Mark Corrigan Offaly 1981 , 1985 Coughlan , Eugene Eugene Coughlan Offaly 1981 , 1985 Coughlan , Ger Ger Coughlan Offaly 1981 , 1985 Coughlan , John John Coughlan Cork 1926 , 1931 Coughlan , Pat Pat Coughlan Cork 1893 , 1894 Coughlan , Tom Tom Coughlan Cork 1902 , 1903 Cronin , Michael Michael Cronin Cork 1893 , 1894 Cross , Micky Micky Cross Limerick 1934 , 1936 Cullinane , Jack Jack Cullinane Cork 1893 , 1894 Cummins , Brendan Brendan Cummins Tipperary 2001 , 2010 Cunningham , Anthony Anthony Cunningham Galway 1987 , 1988 Curran , Ronan Ronan Curran Cork 2004 , 2005 Daly , Anthony Anthony Daly Clare 1995 , 1997 Delaney , Pat Pat Delaney Offaly 1981 , 1985 Delea , James James Delea Cork 1893 , 1894 Dermody , Jim Jim Dermody Kilkenny 1932 , 1933 Desmond , Jerry Jerry Desmond Cork 1902 , 1903 Dooley , Billy Billy Dooley Offaly 1994 , 1998 Dooley , Johnny Johnny Dooley Offaly 1994 , 1998 Dowling , Liam Liam Dowling Cork 1952 , 1953 Doyle , Eddie Eddie Doyle Kilkenny 1932 , 1933 Doyle , Liam Liam Doyle Clare 1995 , 1997 Drew , Dan Dan Drew Cork 1890 , 1892 Duggan , Jack Jack Duggan Kilkenny 1933 , 1935 Dunne , Claus Claus Dunne Kilkenny 1967 , 1969 Dunne , Johnny Johnny Dunne Kilkenny 1933 , 1935 Dunne , Watty Watty Dunne Tipperary 1898 , 1899 Dwyer , Billy Billy Dwyer Kilkenny 1957 , 1963 Dwyer , Pat Pat Dwyer Kilkenny 1992 , 1993 English , Nicky Nicky English Tipperary 1989 , 1991 Fenton , John John Fenton Cork 1984 , 1986 Finnerty , Pete Pete Finnerty Galway 1987 , 1988 Finlay , Tom Tom Finlay Laois Dublin 1915 , 1924 Fitzgerald , Davy Davy Fitzgerald Clare 1995 , 1997 Fitzgerald , Ger Ger Fitzgerald Cork 1986 , 1990 Flanagan , Jim Jim Flanagan Tipperary 1895 , 1896 Flanagan , Thady Thady Flanagan Tipperary 1895 , 1896 Fleury , Pat Pat Fleury Offaly 1981 , 1985 Foley , Art Art Foley Wexford 1955 , 1956 Fogarty , Aidan Aidan Fogarty Offaly 1981 , 1985 Fox , Pat Pat Fox Tipperary 1989 , 1991 Gardiner , John John Gardiner Cork 2004 , 2005 Gaynor , Len Len Gaynor Tipperary 1965 , 1971 Gleeson , Bill Bill Gleeson Tipperary 1899 , 1900 Gleeson , Billy Billy Gleeson Tipperary 1899 , 1900 Gleeson , Jack Jack Gleeson Tipperary 1899 , 1900 Gleeson , Jack Jack Gleeson Tipperary 1906 , 1908 Gleeson , Tim Tim Gleeson Tipperary 1906 , 1908 Gleeson , Willie Willie Gleeson Limerick 1918 , 1921 Gough , Oliver Oliver Gough Wexford Kilkenny 1955 , 1963 Grace , Dick Dick Grace Kilkenny 1905 , 1907 Grace , Paddy Paddy Grace Kilkenny 1939 , 1947 Hanamy , Martin Martin Hanamy Offaly 1994 , 1998 Hartnett , Josie Josie Hartnett Cork 1953 , 1954 Hartnett , Pat Pat Hartnett Cork 1984 , 1986 Hayes , David David Hayes Cork 1893 , 1894 Hayes , Derry Derry Hayes Cork 1953 , 1954 Healy , Paddy Paddy Healy Cork 1944 , 1946 Hearne , S\\u00e9amus S\\u00e9amus Hearne Wexford 1955 , 1956 Heaslip , Denis Denis Heaslip Kilkenny 1957 , 1963 Heffernan , Christy Christy Heffernan Kilkenny 1982 , 1983 Hegarty , Fergal Fergal Hegarty Clare 1995 , 1997 Hegarty , Steve Steve Hegarty Cork 1893 , 1894 Hennessy , Bill Bill Hennessy Kilkenny 1992 , 1993 Herity , David David Herity Kilkenny 2011 , 2012 Hogan , Ken Ken Hogan Tipperary 1989 , 1991 Horan , P\\u00e1draig P\\u00e1draig Horan Offaly 1981 , 1985 Hough , Willie Willie Hough Limerick 1918 , 1921 Hoyne , Jack Jack Hoyne Kilkenny 1904 , 1905 Humphreys , Jimmy Jimmy Humphreys Limerick 1918 , 1921 Keady , Tony Tony Keady Galway 1987 , 1988 Keane , Jack Jack Keane Limerick 1918 , 1921 Keating , Michael Michael Keating Tipperary 1964 , 1971 Keeshan , Brendan Brendan Keeshan Offaly 1981 , 1985 Kelleher , Jamesy Jamesy Kelleher Cork 1902 , 1903 Kelly , Eoin Eoin Kelly Tipperary 2001 , 2010 Kelly , Jimmy Jimmy Kelly Kilkenny 1939 , 1947 Kelly , Joe Joe Kelly Cork 1944 , 1946 Kelly , Joachim Joachim Kelly Offaly 1981 , 1985 Kenna , Tom Tom Kenna Tipperary 1906 , 1908 Kennedy , Jimmy Jimmy Kennedy Tipperary 1949 , 1950 Kennedy , Martin Martin Kennedy Tipperary 1925 , 1930 Kennefick , Mick Mick Kennefick Cork 1942 , 1943 Kenny , Tom Tom Kenny Kilkenny 1904 , 1907 Kenny , Tom Tom Kenny Cork 2004 , 2005 Kerwick , Tom Tom Kerwick Tipperary 1906 , 1908 Kiely , Larry Larry Kiely Tipperary 1964 , 1965 Kilkenny , Ollie Ollie Kilkenny Galway 1987 , 1988 Kilkenny , Tony Tony Kilkenny Galway 1987 , 1988 Kinahan , Kevin Kevin Kinahan Offaly 1994 , 1998 Langton , Jim Jim Langton Kilkenny 1939 , 1947 Lanigan , Dinny Dinny Lanigan Limerick 1918 , 1921 Lanigan , Jimmy Jimmy Lanigan Tipperary 1930 , 1937 Larkin , Philly Philly Larkin Kilkenny 2000 , 2002 Lawlor , Jim Jim Lawlor Kilkenny 1904 , 1905 Lawlor , Martin Martin Lawlor Kilkenny 1904 , 1905 Leahy , John John Leahy Tipperary 1989 , 1991 Leahy , Johnny Johnny Leahy Tipperary 1916 , 1925 Leahy , Paddy Paddy Leahy Tipperary 1916 , 1925 Leahy , Terry Terry Leahy Kilkenny 1939 , 1947 Lohan , Brian Brian Lohan Clare 1995 , 1997 Lohan , Frank Frank Lohan Clare 1995 , 1997 Lynskey , Brendan Brendan Lynskey Galway 1987 , 1988 Maher , Paddy Paddy Maher Tipperary 1900 , 1906 Mahon , Steve Steve Mahon Galway 1980 , 1987 Malone , Mick Mick Malone Cork 1976 , 1977 Malone , Pat Pat Malone Galway 1987 , 1988 Martin , Kevin Kevin Martin Offaly 1994 , 1998 McCarthy , Liam Liam McCarthy Kilkenny 1992 , 1993 McCarthy , Niall Niall McCarthy Cork 2004 , 2005 McCarthy , Teddy Teddy McCarthy Cork 1986 , 1990 Won a football medal in 1990 , a unique double . McCarthy , Tommy Tommy McCarthy Limerick 1934 , 1936 McConkey , Bob Bob McConkey Limerick 1918 , 1921 McCormack , Tom Tom McCormack Kilkenny 1974 , 1975 McCormack , Tom Tom McCormack Kilkenny 1911 , 1912 McDonnell , Pat Pat McDonnell Cork 1970 , 1976 McEvoy , Brian Brian McEvoy Kilkenny 2000 , 2002 McGovern , Johnny Johnny McGovern Kilkenny 1957 , 1963 McGrath , Dave Dave McGrath Cork 1902 , 1903 McGrath , Michael Michael McGrath Galway 1987 , 1988 McGrath , Tom Tom McGrath Limerick 1918 , 1921 McInerney , Gerry Gerry McInerney Galway 1987 , 1988 McInerney , Pa Pa McInerney Clare Dublin 1914 , 1927 McInerney , Paddy Paddy McInerney Limerick 1918 , 1921 McMahon , Paddy Paddy McMahon Limerick 1936 , 1940 McMahon , Se\\u00e1nie Se\\u00e1nie McMahon Clare 1995 , 1997 Mockler , Jack Jack Mockler Tipperary 1906 , 1908 Molloy , P.J. P.J. Molloy Galway 1980 , 1987 Moloughney , Tom Tom Moloughney Tipperary 1961 , 1962 Moore , Tommy Tommy Moore Dublin 1917 , 1920 Morrissey , Eamon Eamon Morrissey Kilkenny 1992 , 1993 Mulcahy , Denis Denis Mulcahy Cork 1984 , 1986 Mulcahy , Jack Jack Mulcahy Kilkenny 1939 , 1947 Murphy , Brian Brian Murphy Cork 2004 , 2005 Murphy , Kieran Kieran Murphy Cork 2004 , 2005 Murphy , Michael Michael Murphy Cork 1893 , 1894 Murphy , Mick Mick Murphy Cork 1919 , 1926 Murphy , Mick Mick Murphy Limerick 1918 , 1921 Murphy , Tom Tom Murphy Kilkenny 1963 , 1969 Naughton , Martin Martin Naughton Galway 1987 , 1988 Neville , Mick Mick Neville Dublin Limerick 1917 , 1920 Neville , Tom Tom Neville Wexford 1960 , 1968 Nolan , Pat Pat Nolan Wexford 1960 , 1968 Norberg , Jer Jer Norberg Cork 1893 , 1894 O\'Brien , Donal Donal O\'Brien Tipperary 1961 , 1962 O\'Brien , Jim Jim O\'Brien Tipperary 1906 , 1908 O\'Brien , Jimmy Jimmy O\'Brien Wexford 1960 , 1968 O\'Brien , Martin Martin O\'Brien Tipperary 1906 , 1908 O\'Connell , Donie Donie O\'Connell Tipperary 1989 , 1991 O\'Connell , P.J. P.J. O\'Connell Clare 1995 , 1997 O\'Connell , Willie John Willie John O\'Connell Cork 1893 , 1894 O\'Connor , Eddie Eddie O\'Connor Kilkenny 1992 , 1993 O\'Connor , Jamesie Jamesie O\'Connor Clare 1995 , 1997 O\'Connor , Jerry Jerry O\'Connor Cork 2004 , 2005 O\'Donnell , Arthur Arthur O\'Donnell Tipperary 1916 , 1925 O\'Donoghue , \\u00c9amonn \\u00c9amonn O\'Donoghue Cork 1976 , 1978 O\'Donoghue , John John O\'Donoghue Tipperary 1964 , 1965 O\'Dwyer , Willie Willie O\'Dwyer Kilkenny Kerry 2006 , 2007 O\'Gara , Matt Matt O\'Gara Tipperary 1961 , 1962 O\'Halloran , Michael Michael O\'Halloran Clare 1995 , 1997 O\'Keeffe , Denis Denis O\'Keeffe Cork 1902 , 1903 O\'Keeffe , Dick Dick O\'Keeffe Tipperary 1898 , 1899 O\'Keeffe , Jim Jim O\'Keeffe Tipperary 1898 , 1899 O\'Keeffe , Joe Joe O\'Keeffe Tipperary 1900 , 1906 O\'Leary , Jerh Jerh O\'Leary Cork 1902 , 1903 O\'Leary , John John O\'Leary Cork 1893 , 1894 O\'Loughlin , Ger Ger O\'Loughlin Clare 1995 , 1997 O\'Neill , Dinny Dinny O\'Neill Dublin 1924 , 1927 O\'Neill , Pat Pat O\'Neill Kilkenny 1992 , 1993 O\'Neill , Willie Willie O\'Neill Cork 1902 , 1903 O\'Riordan , Mossy Mossy O\'Riordan Cork 1946 , 1952 O\'Sullivan , Ted Ted O\'Sullivan Cork 1941 , 1943 O\'Sullivan , Tom Tom O\'Sullivan Cork 1953 , 1954 Orr , Nicky Nicky Orr Kilkenny 1974 , 1975 Owens , Danny Danny Owens Offaly 1981 , 1985 Phelan , Michael Michael Phelan Kilkenny 1992 , 1993 Pilkington , Johnny Johnny Pilkington Offaly 1994 , 1998 Power , Jack Jack Power Tipperary 1916 , 1925 Power , Jackie Jackie Power Limerick 1936 , 1940 Power , John John Power Kilkenny 2014 , 2015 Power , Martin Martin Power Kilkenny 1932 , 1933 Power , Richie Richie Power Kilkenny 1982 , 1983 Rackard , Bobby Bobby Rackard Wexford 1955 , 1956 Rackard , Nicky Nicky Rackard Wexford 1955 , 1956 Rice , Michel Michel Rice Kilkenny 2009 , 2011 Rigney , Hubert Hubert Rigney Offaly 1994 , 1998 Riordan , Steva Steva Riordan Cork 1902 , 1903 Ronan , Adrian Adrian Ronan Kilkenny 1992 , 1993 Ronan , Neil Neil Ronan Cork 1999 , 2005 Ryall , James James Ryall Kilkenny 2003 , 2006 Ryan , Aidan Aidan Ryan Tipperary 1989 , 1991 Ryan , Bobby Bobby Ryan Tipperary 1989 , 1991 Ryan , \\u00c9anna \\u00c9anna Ryan Galway 1987 , 1988 Ryan , Ed Ed Ryan Tipperary 1896 , 1898 Ryan , John John Ryan Dublin 1917 , 1920 Ryan , John John Ryan Tipperary 1898 , 1899 Ryan , Mick Mick Ryan Limerick 1934 , 1936 Ryan , Ned Ned Ryan Tipperary 1950 , 1951 Ryan , Paddy Paddy Ryan Tipperary 1937 , 1945 Ryan , Tom Tom Ryan Tipperary 1961 , 1962 Ryan , Tom Tom Ryan Wexford 1955 , 1956 Ryan , Tommy Tommy Ryan Tipperary 1949 , 1950 Ryan , Willie Willie Ryan Limerick 1918 , 1921 Scanlon , Paddy Paddy Scanlon Limerick 1936 , 1940 Scannell , Denis Denis Scannell Cork 1892 , 1893 Sheehy , Noel Noel Sheehy Tipperary 1989 , 1991 Sherlock , Wayne Wayne Sherlock Cork 1999 , 2004 Simpson , Liam Liam Simpson Kilkenny 1992 , 1993 Tobin , Ned Ned Tobin Dublin 1920 , 1927 Treacy , Tommy Tommy Treacy Tipperary 1930 , 1937 Troy , Jim Jim Troy Offaly 1985 , 1994 Troy , John John Troy Offaly 1994 , 1998 Tuohy , Fergus Fergus Tuohy Clare 1995 , 1997 Twomey , Joe Joe Twomey Cork 1952 , 1953 Walsh , Denis Denis Walsh Cork 1986 , 1990 Walsh , Michael Michael Walsh Kilkenny 1992 , 1993 Walsh , P\\u00e1draig P\\u00e1draig Walsh Kilkenny 2014 , 2015 Walsh , Tom Tom Walsh Kilkenny 1963 , 1967 Whelehan , Brian Brian Whelehan Offaly 1994 , 1998 Young , Jim Jim Young Cork 1893 , 1894 Abernethy , Liam Liam Abernethy Cork 1952 Aherne , Dominic Dominic Aherne Wexford 1955 Allen , John John Allen Cork 1978 Banim , Willie Willie Banim Dublin 1924 Barrett , Jack Jack Barrett Cork 1941 Barrett , Ned Ned Barrett London 1901 Barron , John John Barron Waterford 1959 Barry , Jim Jim Barry London 1901 Barry , Pat Pat Barry Cork 1976 Barry , Paddy Paddy Barry Limerick 1918 Barry , Se\\u00e1nie Se\\u00e1nie Barry Cork 1966 Barry , Tom Tom Barry London 1901 Barry - Murphy , John John Barry - Murphy Cork 1919 Baston , Vin Vin Baston Waterford 1948 Beckett , Derry Derry Beckett Cork 1942 Won a medal as a non-playing substitute in 1941 Belton , Tom Tom Belton Dublin 1889 Bennett , Jim Jim Bennett Kilkenny 1967 Bennett , John John Bennett Cork 1966 Bennis , Phil Phil Bennis Limerick 1973 Bennis , Richie Richie Bennis Limerick 1973 Bernie , Dave Dave Bernie Wexford 1968 Berry , Jack Jack Berry Wexford 1968 Bishop , John John Bishop Dublin 1889 Blanchfield , Dick Dick Blanchfield Kilkenny 1967 Bowe , Ned Ned Bowe Tipperary 1887 Brannigan , Bobby Bobby Brannigan Kilkenny 1939 Brazil , Tom Tom Brazil Limerick 1897 Brennan , Canice Canice Brennan Kilkenny 2000 Brennan , Dick Dick Brennan Kilkenny 1904 Brennan , Martin Martin Brennan Kilkenny 1969 Brophy , Mick Mick Brophy Kilkenny 1957 Brophy , Mick Mick Brophy Dublin 1938 Browne , Alan Alan Browne Cork 1999 Browne , Richard Richard Browne Cork 1986 Buckley , Connie Connie Buckley Cork 1941 Buckley , John John Buckley Cork 1890 Buckley , Se\\u00e1n Se\\u00e1n Buckley Kilkenny 1969 Buggy , Paddy Paddy Buggy Kilkenny 1957 Bugler , Brendan Brendan Bugler Clare 2013 Burke , Billy Billy Burke Kilkenny 1939 Burke , Frank Frank Burke Galway 1980 Burke , Gerry Gerry Burke Galway 1988 Burke , Hugh Hugh Burke Dublin 1917 Burke , Tom Tom Burke Tipperary 1887 Butler , Mick Mick Butler Dublin 1938 Butler , Patrick Patrick Butler Limerick 1897 Butler , Tom Tom Butler Dublin 1889 Byrne , Billy Billy Byrne Wexford Byrne , Denis Denis Byrne Kilkenny Tipperary 2000 Byrne , Jim Jim Byrne Dublin 1938 Byrne , Locky Locky Byrne Kilkenny Waterford 1935 Byrne , Ned Ned Byrne Kilkenny 1972 Byrne , Paul Paul Byrne Tipperary 1971 Byrne , Stephen Stephen Byrne Offaly 1998 Cahill , J. J. Cahill Dublin 1889 Cahill , Michael Michael Cahill Tipperary Cahill , Willie Willie Cahill Kilkenny 1947 Callanan , S\\u00e9amus S\\u00e9amus Callanan Tipperary Campion , Paddy Paddy Campion Laois 1915 Carew , Anthony Anthony Carew Tipperary 1908 Carew , Eddie Eddie Carew Waterford 1948 Carroll , Jim Jim Carroll Laois 1915 Carroll , Joe Joe Carroll Laois 1915 Carroll , John John Carroll Tipperary 2001 Carroll , John John Carroll Kilkenny 1932 Carroll , John John Carroll Laois 1915 Carroll , Mick Mick Carroll Tipperary 1887 Carroll , Pat Pat Carroll Offaly 1981 Carroll , Pat Pat Carroll Kilkenny 1967 Carroll , Tom Tom Carroll Tipperary 1887 Carroll , Tommy Tommy Carroll Kilkenny 1922 Casey , Cathal Cathal Casey Cork 1990 Casserly , Michael Michael Casserly Cork 1892 Cheasty , Tom Tom Cheasty Waterford 1959 Clancy , Jim Jim Clancy Clare 1914 Clancy , Willie Willie Clancy Cork 1931 Clarke , Ned Ned Clarke Limerick 1940 Cleary , Pat Pat Cleary Offaly 1985 Clifford , Donal Donal Clifford Cork 1970 Clifford , Johnny Johnny Clifford Cork 1954 Won a medal as a non-playing substitute in 1953 Close , Jim Jim Close Limerick 1934 Clune , Jim Jim Clune Dublin 1920 Codd , Martin Martin Codd Wexford 1956 Codd , Paul Paul Codd Wexford Coen , S\\u00e9amus S\\u00e9amus Coen Galway 1980 Coffey , Jimmy Jimmy Coffey Tipperary 1937 Coffey , John John Coffey Tipperary 1945 Coleman , Michael Michael Coleman Galway 1988 Colfer , Ned Ned Colfer Wexford 1968 Collins , P\\u00e1draic P\\u00e1draic Collins Clare 2013 Collison , Jer Jer Collison Tipperary 1916 Condon , Jackie Jackie Condon Waterford 1959 Condon , John John Condon Limerick 1897 Conlon , John John Conlon Clare 2013 Conneely , Michael Michael Conneely Galway 1980 Conneely , Tom Tom Conneely Offaly 1985 Connolly , Joe Joe Connolly Galway 1980 Connolly , John John Connolly Galway 1980 Connolly , Michael Michael Connolly Galway 1980 Connolly , Tony Tony Connolly Cork 1966 Considine , John John Considine Cork 1990 Considine , Willie Willie Considine Clare 1914 Conway , James James Conway Cork 1892 Cooke , Tommy Tommy Cooke Limerick 1940 Cooney , Jimmy Jimmy Cooney Galway 1980 Cooney , Jimmy Jimmy Cooney Tipperary 1937 Corcoran , \\u00c9amonn \\u00c9amonn Corcoran Tipperary 2001 Corcoran , Pat Pat Corcoran Wexford 1910 Corrigan , Paddy Paddy Corrigan Offaly 1985 Costello , Thomas Thomas Costello Tipperary 2001 Cottrill , Jim Jim Cottrill Limerick 1897 Coughlan , Dan Dan Coughlan Cork 1894 Coughlan , Denis Denis Coughlan Cork 1892 Coughlan , Frank Frank Coughlan Dublin 1889 Coughlan , Jack Jack Coughlan London 1901 Coughlan , Jerh Jerh Coughlan Cork 1903 Cregan , \\u00c9amonn \\u00c9amonn Cregan Limerick 1973 Cregan , Ned Ned Cregan Limerick 1934 Cregan , Peter Peter Cregan Limerick 1940 Cronin , Mick Mick Cronin Tipperary 1930 Crowley , C. C. Crowley London 1901 Crowley , Frank Frank Crowley Kerry 1891 Crowley , James James Crowley Kerry 1891 Cullinane , Charlie Charlie Cullinane Cork 1970 Cummins , Brendan Brendan Cummins Cork 1976 Cummins , Mike Mike Cummins Wexford 1910 Cunningham , Tom Tom Cunningham Waterford 1959 Currams , Liam Liam Currams Offaly 1981 Curran , Paul Paul Curran Tipperary Curran , Tom Tom Curran Waterford 1948 Cusack , John John Cusack Waterford 1948 Cushe , Ger Ger Cushe Wexford Daly , Ed Ed Daly Waterford 1948 Daly , Jack Jack Daly Laois 1915 Daly , Johnny Johnny Daly Cork 1902 Daly , William William Daly Cork 1902 Daniels , Mick Mick Daniels Dublin 1938 Darcy , Jack Jack Darcy Tipperary 1925 Darcy , Mick Mick Darcy Tipperary 1925 Delaney , P.J. P.J. Delaney Kilkenny 1993 Delaney , Paul Paul Delaney Tipperary 1991 Delahunty , Joe Joe Delahunty Kilkenny 1909 Dempsey , Tom Tom Dempsey Wexford Derivan , Mick Mick Derivan Galway 1923 Devereux , William William Devereux Wexford 1910 Devitt , Jim Jim Devitt Tipperary 1945 Dillon , Cian Cian Dillon Clare 2013 Dixon , Tom Tom Dixon Wexford 1956 Doherty , Jack Jack Doherty Tipperary 1916 Doherty , Paddy Paddy Doherty Tipperary 1896 Donegan , Jim Jim Donegan Kilkenny 1947 Donnellan , Patrick Patrick Donnellan Clare 2013 Donoghue , Paddy Paddy Donoghue Kilkenny 1922 Donoghue , Tom Tom Donoghue Offaly 1981 Donohoe , Sim Sim Donohoe Wexford 1910 Doody , Pat Pat Doody Dublin 1938 Doody , Tim Tim Doody London 1901 Doolan , Peter Peter Doolan Cork 1966 Doran , Tony Tony Doran Wexford 1968 Dowling , Mossie Mossie Dowling Limerick 1973 Dowling , Se\\u00e1n Se\\u00e1n Dowling Kilkenny 2003 Downes , Michael Michael Downes Limerick 1897 Downey , Shem Shem Downey Kilkenny 1947 Doyle , Dick Dick Doyle Wexford 1910 Duffy , Tom Tom Duffy Tipperary 1925 Duignan , Michael Michael Duignan Offaly 1998 Dunne , Benny Benny Dunne Tipperary Dunne , Dan Dan Dunne Kilkenny 1932 Dunne , Jack Jack Dunne Tipperary 1887 Dunne , Jim Jim Dunne Kilkenny 1904 Dunne , Liam Liam Dunne Wexford Dunne , Tom Tom Dunne Kerry 1891 Dunne , Tommy Tommy Dunne Tipperary 2001 Dunphy , Jack Jack Dunphy Laois 1915 Dunphy , Jim Jim Dunphy Kilkenny 1909 Dunphy , Ned Ned Dunphy Kilkenny 1922 Dunphy , Wattie Wattie Dunphy Kilkenny 1922 Dwan , Tom Tom Dwan Tipperary 1916 Dwyer , Jer Jer Dwyer Tipperary 1887 Dwyer , Tim Tim Dwyer Tipperary 1887 Enright , Eddie Eddie Enright Tipperary 2001 Errity , Joe Joe Errity Offaly 1998 Fahy , Donnacha Donnacha Fahy Tipperary 2001 Fahy , Ned Ned Fahy Dublin 1927 Fanning , Declan Declan Fanning Tipperary Farrell , Phil Phil Farrell Dublin 1938 Fenlon , Adrian Adrian Fenlon Wexford Fennelly , Kevin Kevin Fennelly Kilkenny 1979 Fielding , Pat Pat Fielding Kilkenny 1904 Finlay , Jack Jack Finlay Laois Dublin 1915 Finn , John John Finn Limerick 1897 Finn , Mick Mick Finn Limerick 1897 Finn , Paul Paul Finn Wexford Fitzgerald , J. J. Fitzgerald London 1901 Fitzgerald , John John Fitzgerald Tipperary 1908 Fitzgerald , Paddy Paddy Fitzgerald Cork 1966 Fitzgerald , Pat Pat Fitzgerald Tipperary 1908 Fitzgibbon , John John Fitzgibbon Cork 1990 Fitzgibbon , William William Fitzgibbon Cork 1902 Fitzhenry , Damien Damien Fitzhenry Wexford Fitzmaurice , Maurice Maurice Fitzmaurice Kerry 1891 Fitzmaurice , Paudie Paudie Fitzmaurice Limerick 1973 Fitzpatrick , Joe Joe Fitzpatrick Tipperary 1916 Fitzpatrick , John John Fitzpatrick Kilkenny 1933 Flanagan , Jack Jack Flanagan Tipperary 1899 Flanagan , John John Flanagan Tipperary 1971 Flaherty , Johnny Johnny Flaherty Offaly 1981 Flaherty , Larry Larry Flaherty Cork 1903 Flaherty , Pat Pat Flaherty Cork 1893 Flanagan , Mick Mick Flanagan Clare 1914 Flannelly , Mick Mick Flannelly Waterford 1959 Fleming , Andy Andy Fleming Waterford 1948 Fleming , Tom Tom Fleming Galway 1923 Flood , Jim Jim Flood Limerick 1897 Flynn , Austin Austin Flynn Waterford 1959 Flynn , Maurice Maurice Flynn Limerick 1897 Flynn , Mick Mick Flynn Dublin 1938 Flynn , Patrick Patrick Flynn Limerick 1897 Fogarty , Declan Declan Fogarty Offaly 1985 Foley , Mark Mark Foley Cork 1990 Foley , Se\\u00e1n Se\\u00e1n Foley Limerick 1973 Forde , Bernie Bernie Forde Galway 1980 Forde , Christy Christy Forde Dublin 1938 Forde , David David Forde Clare 1997 Fortune , Dick Dick Fortune Wexford 1910 Fortune , Jas Jas Fortune Wexford 1910 Fox , John John Fox Clare 1914 Galvin , Colm Colm Galvin Clare 2013 Galvin , Willie Willie Galvin Waterford 1948 Gantley , Finbarr Finbarr Gantley Galway 1980 Gargan , Jack Jack Gargan Kilkenny Dublin 1947 Garvey , Jimmy Jimmy Garvey Galway 1923 Gibbs , Bernie Bernie Gibbs Galway 1923 Gill , Mick Mick Gill Dublin 1938 Gilligan , Ned Ned Gilligan Dublin 1889 Gilligan , Niall Niall Gilligan Clare 1997 Gilmartin , Ed Ed Gilmartin Galway 1923 Gilmartin , Jim Jim Gilmartin Dublin 1938 Gleeson , Conor Conor Gleeson Tipperary 2001 Gleeson , Eddie Eddie Gleeson Tipperary 1945 Gleeson , Jack Jack Gleeson Tipperary 1937 Gleeson , Jack Jack Gleeson Dublin 1927 Gleeson , Jim Jim Gleeson Tipperary 1895 Gleeson , John John Gleeson Tipperary 1971 Glendon , Pat Pat Glendon Kilkenny 1922 Goldsboro , Harry Harry Goldsboro Tipperary 1945 Good , Jackie Jackie Good Waterford 1948 Goode , Tom Tom Goode Cork 1890 Goulding , \\u00c9amonn \\u00c9amonn Goulding Cork 1954 Grace , Dan Dan Grace Kilkenny 1904 Grace , Ned Ned Grace Clare 1914 Gray , Harry Harry Gray Laois Dublin 1938 Grey , Ned Ned Grey Cork 1919 Griffin , Willie Willie Griffin Cork 1952 Grimes , Denis Denis Grimes Limerick 1897 Grimes , \\u00c9amonn \\u00c9amonn Grimes Limerick 1973 Grimes , Phil Phil Grimes Waterford 1959 Won a medal as a non-playing substitute in 1948 Guerin , Jim Jim Guerin Clare 1914 Guinan , Larry Larry Guinan Waterford 1959 Guiney , Rod Rod Guiney Wexford Hackett , Charles Charles Hackett Dublin 1889 Hackett , Martin Martin Hackett Dublin 1917 Hackett , Stephen Stephen Hackett Tipperary 1925 Halloran , Denis Denis Halloran Cork 1892 Hanley , James James Hanley Limerick 1897 Hanniffy , Darren Darren Hanniffy Offaly 1998 Hanniffy , Gary Gary Hanniffy Offaly 1998 Harding , Jack Jack Harding Wexford 1960 Harney , Ignatius Ignatius Harney Galway 1923 Harney , Jimmy Jimmy Harney Tipperary 1930 Harney , Joe Joe Harney Waterford 1959 Harper , James James Harper Dublin 1889 Hartigan , Bernie Bernie Hartigan Limerick 1973 Hartigan , Pat Pat Hartigan Limerick 1973 Hassett , Jim Jim Hassett Cork 1919 Hassett , Matt Matt Hassett Tipperary 1961 Hayden , Paddy Paddy Hayden Kilkenny 1947 Hayes , Jer Jer Hayes Tipperary 1906 Hayes , Joe Joe Hayes Tipperary 1989 Hayes , John Joe John Joe Hayes Tipperary 1925 Hayes , Mick Mick Hayes Waterford 1948 Hayes , Ned Ned Hayes Tipperary 1900 Hayes , Paddy Paddy Hayes Tipperary 1900 Hayes , Stephen Stephen Hayes Cork 1894 Hayes , Tom Tom Hayes Dublin 1920 Healy , Tom Tom Healy Tipperary 1887 Heaney , James James Heaney Tipperary 1930 Heffernan , Jimmy Jimmy Heffernan Kilkenny 1947 Heffernan , John John Heffernan Tipperary 1989 Hegarty , Pat Pat Hegarty Cork 1970 Henchion , Jer Jer Henchion Cork 1890 Hennebry , Bill Bill Hennebry Kilkenny 1909 Hennessy , Bill Bill Hennessy Cork 1903 Hennessy , S\\u00e9amus S\\u00e9amus Hennessy Tipperary Herbert , Tom Tom Herbert Limerick 1940 Hickey , Mick Mick Hickey Limerick 1940 Hickey , Mick Mick Hickey Waterford 1948 Higgins , Bill Bill Higgins Cork 1926 Hiney , Jack Jack Hiney Laois 1915 Hinks , Bobby Bobby Hinks Kilkenny 1939 Hodgins , John John Hodgins Cork 1984 Hogan , S\\u00e9amus S\\u00e9amus Hogan Tipperary 1971 Holland , Mick Mick Holland Dublin 1924 Holohan , John John Holohan Kilkenny 1922 Honan , Darach Darach Honan Clare 2013 Horgan , Dan Dan Horgan London 1901 Horgan , Denis Denis Horgan Cork 1890 Horgan , M. M. Horgan London 1901 Horgan , Michael Michael Horgan Cork 1890 Horgan , S\\u00e9amus S\\u00e9amus Horgan Limerick 1973 Hyde , Se\\u00e1n Se\\u00e1n Hyde Cork Dublin 1917 Hynes , John John Hynes Limerick 1897 Irwin , Tom Tom Irwin Cork 1892 Jacob , Christy Christy Jacob Wexford 1968 Kavanagh , Dave Dave Kavanagh Wexford 1910 Kavanagh , Ned Ned Kavanagh Kilkenny 1947 Kavanagh , Pat Pat Kavanagh Kilkenny 1969 Keane , Larry Larry Keane Tipperary 1958 Kearney , D.J. D.J. Kearney Cork 1926 Keegan , James James Keegan Cork 1892 Kehoe , Andy Andy Kehoe Wexford 1910 Kehoe , Colm Colm Kehoe Wexford Kehoe , Paddy Paddy Kehoe Wexford 1955 Kelleher , Frank Frank Kelleher Cork 1919 Kelleher , Jim Jim Kelleher Cork 1894 Kelleher , John John Kelleher Cork 1890 Kelleher , Tim Tim Kelleher Cork 1890 Kelliher , Jer Jer Kelliher London 1901 Kelly , Andy Andy Kelly Galway 1923 Kelly , Brendan Brendan Kelly Offaly 1994 Kelly , Eddie Eddie Kelly Wexford 1968 Kelly , John John Kelly Tipperary 1971 Kelly , Maurice Maurice Kelly Kerry 1891 Kelly , Mickey Mickey Kelly Kilkenny 1957 Kelly , Patrick Patrick Kelly Clare 2013 Kelly , Paul Paul Kelly Tipperary 2001 Kelly , Tommy Tommy Kelly Dublin 1924 Kelly , Tony Tony Kelly Clare 2013 Kenneally , John John Kenneally Cork 1892 Kenneally , Johnny Johnny Kenneally Cork 1929 Kennedy , Dan Dan Kennedy Kilkenny 1947 Kennedy , David David Kennedy Tipperary 2001 Kennedy , Eamonn Eamonn Kennedy Kilkenny 2000 Kennedy , Jimmy Jimmy Kennedy Cork 1919 Won a medal as a non-playing substitute in 1926 Kennedy , John John Kennedy Tipperary 1989 Kennedy , John John Kennedy Wexford 1960 Kenny , Bill Bill Kenny Kilkenny 1922 Kenny , Mick Mick Kenny Galway 1923 Kenny , Mick Mick Kenny Kilkenny 1957 Kennedy , Paddy Paddy Kennedy Limerick 1918 Kennefick , Paddy Paddy Kennefick Dublin 1917 Kenny , Stephen Stephen Kenny Tipperary 1925 Kennedy , Tom Tom Kennedy Tipperary 1937 Keoghan , Liam Liam Keoghan Kilkenny 1993 Kerwick , Bill Bill Kerwick Tipperary 1895 Kerwick , Dan Dan Kerwick Dublin 1889 Kidney , Dinny Dinny Kidney Cork 1903 Kidney , John John Kidney Cork 1894 Kiely , John John Kiely Waterford 1959 Won a medal as a non-playing substitute in 1948 King , John John King London 1901 King , Martin Martin King Galway 1923 King , Paddy Paddy King London 1901 Kingston , Kieran Kieran Kingston Cork 1986 Kinsella , John John Kinsella Kilkenny 1972 King , Liam Liam King Tipperary 1971 Kirby , Michael Michael Kirby Kerry 1891 Kirwan , Paddy Paddy Kirwan Offaly 1981 Kissane , Richard Richard Kissane Kerry 1891 Lacey , Mick Mick Lacey Waterford 1959 Laffan , Gary Gary Laffan Wexford Lambe , John John Lambe Dublin 1889 Lambe , Ned Ned Lambe Tipperary 1887 Landers , Mark Mark Landers Cork 1999 Lane , Dan Dan Lane Cork 1890 Larkin , Tom Tom Larkin Tipperary 1958 Lawler , Mick Mick Lawler Kilkenny 1969 Lawlor , Martin Martin Lawlor Kilkenny 1922 Leahy , John John Leahy Cork 1892 Leahy , Pat Pat Leahy Cork 1902 Leahy , Pat Pat Leahy Tipperary 1887 Leahy , Tommy Tommy Leahy Tipperary 1930 Leamy , John John Leamy Tipperary 1887 Lennon , Jack Jack Lennon Kilkenny 1913 Linehan , Dan Dan Linehan Cork 1890 Linehan , John John Linehan Cork 1890 Lonergan , Mick Mick Lonergan Tipperary 1964 Looney , Dan Dan Looney Cork 1890 Looney , S\\u00e9amus S\\u00e9amus Looney Cork 1970 Loughnane , Bill Bill Loughnane Dublin 1938 Loughnane , Francis Francis Loughnane Tipperary 1971 Lucey , Con Con Lucey Cork 1919 Lynam , Jimmy Jimmy Lynam Cork 1952 Lynch , Colin Colin Lynch Clare 1997 Lynch , Jim Jim Lynch Kilkenny 1967 Lynch , Jim Jim Lynch London 1901 Lynch , Paul Paul Lynch Wexford 1968 Lynch , Tim Tim Lynch Cork 1902 Lyons , Cyril Cyril Lyons Clare 1995 Mackessy , Billy Billy Mackessy Cork 1903 Won a football medal in 1911 Mackey , Dan Dan Mackey Tipperary 1937 Mackey , Paddy Paddy Mackey Wexford 1910 Won football medals in 1915 , 1916 , 1917 and 1918 Madigan , Mick Mick Madigan Dublin 1889 Maher , Andy Andy Maher Tipperary 1887 Maher , Billy Billy Maher Tipperary 1900 Maher , Brendan Brendan Maher Tipperary Maher , Jimmy Jimmy Maher Tipperary 1945 Maher , John John Maher Tipperary 1895 Maher , John John Maher Kilkenny 1957 Maher , Matty Matty Maher Tipperary 1887 Maher , Ned Ned Maher Tipperary 1898 Maher , P\\u00e1draic P\\u00e1draic Maher Tipperary Maher , Pat Pat Maher Kilkenny 1904 Maher , Patrick Patrick Maher Tipperary Maher , Peter Peter Maher Tipperary 1895 Maher , Philip Philip Maher Tipperary 2001 Maher , Tom Tom Maher Dublin 1889 Maher , Tom Tom Maher Tipperary 1887 Maher , Tony Tony Maher Cork 1970 Mahony , John John Mahony Kerry 1891 Mahony , Junior Junior Mahony Galway 1923 Mahony , Tom Tom Mahony Cork 1902 Mangan , Tom Tom Mangan Limerick 1921 Marnell , Mark Mark Marnell Kilkenny 1947 Martin , Damien Damien Martin Offaly 1981 McCarthy , Donal Donal McCarthy Cork 1929 McCarthy , Jim Jim McCarthy Limerick 1940 McCarthy , Justin Justin McCarthy Cork 1966 McCarthy , Michael Michael McCarthy Kerry 1891 McCarthy , Rory Rory McCarthy Wexford McCarthy , Se\\u00e1n Se\\u00e1n McCarthy Cork 1990 McCarthy , Thade Donal Thade Donal McCarthy Kerry 1891 McCarthy , Thade Eugene Thade Eugene McCarthy Kerry 1891 McCormack , Fergal Fergal McCormack Cork 1999 McDonald , Mark Mark McDonald Kilkenny 1922 McDonnell , Jim Jim McDonnell Kerry 1891 McDonnell , Mossy Mossy McDonnell Dublin 1938 McEvoy , Ned Ned McEvoy Laois 1915 McGrath , Conor Conor McGrath Clare 2013 McGrath , Leonard Leonard McGrath Galway 1923 McGrath , Noel Noel McGrath Tipperary McGrath , Oliver Oliver McGrath Wexford 1960 McGrath , Se\\u00e1nie Se\\u00e1nie McGrath Cork 1999 McGrath , Shane Shane McGrath Tipperary McGrath , Tom Tom McGrath Clare 1914 McGuckin , Kieran Kieran McGuckin Cork 1990 McGuckin , Shane Shane McGuckin Offaly 1994 McHugh , Bill Bill McHugh Wexford 1910 McInerney , Cathal Cathal McInerney Clare 2013 McInerney , David David McInerney Clare 2013 McInerney , Niall Niall McInerney Galway 1980 McKenna , Joe Joe McKenna Limerick 1973 McKenna , Paul Paul McKenna Tipperary 1930 McKenna , Thomas Thomas McKenna Dublin 1889 McMahon , Charlie Charlie McMahon Dublin 1938 McMahon , M. M. McMahon London 1901 McNamara , Martin Martin McNamara Tipperary 1887 McNamara , Stephen Stephen McNamara Clare 1995 Millea , Joe Joe Millea Kilkenny 1969 Mitchell , John John Mitchell Wexford 1960 Mockler , John John Mockler Tipperary 1887 Mockler , Martin Martin Mockler Tipperary 1925 Moloney , Martin Martin Moloney Clare 1914 Moloughney , Billy Billy Moloughney Tipperary 1961 Moloughney , Joe Joe Moloughney Tipperary 1908 Moore , Willie Willie Moore Limerick 1973 Moore , Willie Willie Moore Cork 1954 Morey , S\\u00e9adna S\\u00e9adna Morey Clare 2013 Moriarty , Fergus Fergus Moriarty Tipperary 1895 Morris , Jimmy Jimmy Morris Galway 1923 Morrissey , Dick Dick Morrissey Galway 1923 Morrissey , \\u00c9amonn \\u00c9amonn Morrissey Kilkenny 1972 Morrissey , Martin \\u00d3g Martin \\u00d3g Morrissey Waterford 1959 Morrisson , Jim Jim Morrisson Cork 1944 Moylan , Christy Christy Moylan Waterford 1948 Mulcahy , Pat Pat Mulcahy Limerick 1897 Mulcahy , Pat Pat Mulcahy Cork 2005 Mulhaire , Paudie Paudie Mulhaire Offaly 1998 Mulhall , John John Mulhall Kilkenny 2011 Mullally , Paddy Paddy Mullally Kilkenny 2003 Mullane , Mick Mick Mullane Limerick 1921 Murnane , Dave Dave Murnane Limerick 1921 Murphy , Barry Barry Murphy Clare 1997 Murphy , Bernie Bernie Murphy Cork 1943 Murphy , Con Con Murphy Cork 1946 Murphy , Denis Denis Murphy Cork 1966 Murphy , Denis Denis Murphy Tipperary 1937 Murphy , Jimmy Jimmy Murphy Tipperary 1916 Murphy , John John Murphy Cork 1893 Murphy , John John Murphy Kerry 1891 Murphy , Kieran Kieran Murphy Cork 2005 Murphy , Larry Larry Murphy Wexford Murphy , Mattie Mattie Murphy Cork 1926 Murphy , Maurice Maurice Murphy Cork 1926 Murphy , Mick Mick Murphy Tipperary 1964 Murphy , Mick Mick Murphy Tipperary 1945 Murphy , Mossy Mossy Murphy Kilkenny 1972 Murphy , Ned Ned Murphy Tipperary 1887 Murphy , Simon Simon Murphy Cork 1970 Murphy , Tadhg Tadhg Murphy Cork 1977 Murphy , Tom Tom Murphy Kilkenny 1993 Murphy , Willie Willie Murphy Kilkenny 1969 Murphy , Willie Willie Murphy Wexford 1968 Murray , Kevin Kevin Murray Cork 1999 Mythen , Jas Jas Mythen Wexford 1910 Nagle , Tim Tim Nagle Cork 1919 Neville , Alan Alan Neville Clare 1995 Neville , Mick Mick Neville Wexford 1910 Nolan , Frankie Frankie Nolan Limerick 1973 Nolan , John John Nolan Wexford 1960 O\'Brien , Conor Conor O\'Brien Tipperary O\'Brien , Eddie Eddie O\'Brien Cork 1970 O\'Brien , J. J. O\'Brien London 1901 O\'Brien , Jim Jim O\'Brien Limerick 1973 O\'Brien , John John O\'Brien Tipperary O\'Brien , John John O\'Brien London 1901 O\'Brien , Paddy Paddy O\'Brien Tipperary 2001 O\'Brien , Patrick Patrick O\'Brien Limerick 1897 O\'Brien , Se\\u00e1n Se\\u00e1n O\'Brien Cork 1952 O\'Brien , Se\\u00e1nie Se\\u00e1nie O\'Brien Kilkenny 1939 O\'Byrne , J.D. J.D. O\'Byrne Dublin 1889 O\'Callaghan , Bill Bill O\'Callaghan Cork 1892 O\'Callaghan , Con Con O\'Callaghan Cork 1892 O\'Carroll , Paddy Paddy O\'Carroll Kerry 1891 O\'Carroll , Paddy Paddy O\'Carroll Limerick 1936 O\'Connell , Jackie Jackie O\'Connell Limerick 1934 O\'Connell , Jer Jer O\'Connell London 1901 O\'Connell , Mickey Mickey O\'Connell Cork 1999 O\'Connell , Nicky Nicky O\'Connell Clare 2013 O\'Connor , George George O\'Connor Wexford O\'Connor , John John O\'Connor Wexford O\'Connor , Johnny Johnny O\'Connor Waterford 1948 O\'Connor , Kevin Kevin O\'Connor Waterford 1948 O\'Connor , Michael Michael O\'Connor Waterford 1959 O\'Connor , Pat Pat O\'Connor Offaly 1994 O\'Connor , Patrick Patrick O\'Connor Clare 2013 O\'Connor , Tadhg Tadhg O\'Connor Tipperary 1971 O\'Connor , Thady Thady O\'Connor Cork 1890 O\'Connor , William William O\'Connor Cork 1892 O\'Donnell , Bill Bill O\'Donnell Tipperary 1937 O\'Donnell , Shane Shane O\'Donnell Clare 2013 O\'Donoghue , Liam Liam O\'Donoghue Limerick 1973 O\'Donoghue , Tom Tom O\'Donoghue Cork 1966 O\'Donovan , Conor Conor O\'Donovan Tipperary 1989 O\'Donovan , Domhnall Domhnall O\'Donovan Clare 2013 O\'Donovan , Se\\u00e1n Se\\u00e1n O\'Donovan Dublin 1917 O\'Dwyer , Billy Billy O\'Dwyer Tipperary 1916 O\'Dwyer , Michael Michael O\'Dwyer Tipperary 1908 O\'Dwyer , Noel Noel O\'Dwyer Tipperary 1971 O\'Dwyer , Paddy Paddy O\'Dwyer Tipperary 1925 O\'Dwyer , Willie Willie O\'Dwyer Tipperary 1916 O\'Gorman , Denis Denis O\'Gorman Tipperary 1937 O\'Gorman , Dick Dick O\'Gorman Cork 1919 O\'Gorman , Larry Larry O\'Gorman Wexford O\'Gorman , Se\\u00e1n Se\\u00e1n O\'Gorman Cork 1990 O\'Grady , Donal Donal O\'Grady Cork 1984 O\'Grady , John John O\'Grady Tipperary 1958 O\'Halloran , John John O\'Halloran Cork 1966 O\'Halloran , Paddy Paddy O\'Halloran Cork 1919 O\'Hanlon , Mick Mick O\'Hanlon Wexford 1955 O\'Keeffe , Bob Bob O\'Keeffe Laois 1915 O\'Keeffe , John John O\'Keeffe Cork 1919 O\'Keeffe , Paddy Paddy O\'Keeffe Cork 1893 O\'Kennedy , Se\\u00e1n Se\\u00e1n O\'Kennedy Wexford 1910 Won football medals in 1915 , 1916 and 1917 O\'Leary , Mark Mark O\'Leary Tipperary 2001 O\'Leary , Michael Michael O\'Leary Cork 1903 O\'Loughlin , Jimmy Jimmy O\'Loughlin Tipperary 1930 O\'Mahony , Conor Conor O\'Mahony Tipperary O\'Neill , Eugene Eugene O\'Neill Tipperary 2001 Riordan , Michael Michael Riordan Kerry 1891 O\'Riordan , Pad Pad O\'Riordan Cork 1890 O\'Riordan , Pat Pat O\'Riordan Cork 1890 O\'Riordan , Pat Pat O\'Riordan Dublin 1889 O\'Riordan , S. S. O\'Riordan Dublin 1889 O\'Rourke , Pat Pat O\'Rourke Kerry 1891 O\'Rourke , Tom Tom O\'Rourke Galway Dublin 1927 O\'Shea , Jackie Jackie O\'Shea Cork 1902 O\'Shea , Maurice Maurice O\'Shea Cork 1902 O\'Shea , Nicholas Nicholas O\'Shea Dublin 1889 O\'Shea , Pat Pat O\'Shea Dublin 1889 O\'Sullivan , Brendan Brendan O\'Sullivan Cork 1990 O\'Sullivan , Jack Jack O\'Sullivan Kerry 1891 O\'Sullivan , Jerry Jerry O\'Sullivan Cork 1966 O\'Sullivan , Michael Michael O\'Sullivan Kerry 1891 O\'Sullivan , Pat Pat O\'Sullivan Cork 1903 O\'Sullivan , Peter Peter O\'Sullivan Tipperary 1971 Ormonde , Paul Paul Ormonde Tipperary 2001 Palmer , F. F. Palmer Dublin 1889 Parfrey , Walter Walter Parfrey Cork 1902 Parker , Michael Michael Parker Wexford 1910 Phelan , Bill Bill Phelan Dublin 1927 Phelan , Conor Conor Phelan Kilkenny 2003 Phelan , Jimmy Jimmy Phelan Kilkenny Carlow 1939 Phelan , Joe Joe Phelan Laois 1915 Pilkington , Declan Declan Pilkington Offaly 1994 Porter , Ned Ned Porter Cork 1942 Power , Amby Amby Power Clare 1914 Power , Joe Joe Power Clare 1914 Power , Jim Jim Power Galway 1923 Power , Ned Ned Power Waterford 1959 Power , Paddy Paddy Power Tipperary 1925 Power , S\\u00e9amus S\\u00e9amus Power Waterford 1959 Power , Se\\u00e1n Se\\u00e1n Power Wexford 1960 Prendergast , Paddy Paddy Prendergast Kilkenny 1947 Prendergast , Shane Shane Prendergast Kilkenny 2015 Purcell , Matt Matt Purcell Tipperary 1900 Purcell , Phil Phil Purcell Tipperary 1930 Purcell , Tommy Tommy Purcell Tipperary 1945 Quane , Jackeen Jackeen Quane Kerry 1891 Quane , Pat Pat Quane Kerry 1891 Quaid , Seamus Seamus Quaid Wexford 1960 Quigley , Dan Dan Quigley Wexford 1968 Quigley , John John Quigley Wexford 1968 Quirke , David David Quirke Cork 1990 Rea , Ned Ned Rea Limerick 1973 Regan , Daith\\u00ed Daith\\u00ed Regan Offaly 1994 Reid , Richie Richie Reid Kilkenny 1979 Reidy , John John Reidy Limerick 1897 Reidy , Liam Liam Reidy Kilkenny 1947 Reilly , E. E. Reilly Cork 1890 Reilly , J. J. Reilly Cork 1890 Ring , Dannix Dannix Ring Cork 1919 Riordan , Dan Dan Riordan Limerick 1897 Riordan , P.J. P.J. Riordan Tipperary 1906 Riordan , Paddy Paddy Riordan Tipperary 1895 Roberts , John John Roberts Kilkenny 1922 Roche , Con Con Roche Cork 1970 Rochford , Mick Mick Rochford Limerick 1918 Roche , Pat Pat Roche Wexford 1910 Rockett , Dick Dick Rockett Kilkenny 1957 Ronayne , Jim Jim Ronayne Cork 1902 Ruskin , Patrick Patrick Ruskin Limerick 1897 Ruth , Matt Matt Ruth Kilkenny 1979 Ryan , Bill Bill Ryan Tipperary 1925 Ryan , Chris Chris Ryan Limerick 1921 Ryan , Colin Colin Ryan Clare 2013 Ryan , Conor Conor Ryan Clare 2013 Ryan , Danny Danny Ryan Tipperary 1887 Ryan , Dick Dick Ryan Limerick 1918 Ryan , Dinny Dinny Ryan Tipperary 1971 Ryan , Fergal Fergal Ryan Cork 1999 Ryan , Gear\\u00f3id Gear\\u00f3id Ryan Tipperary Ryan , Harry Harry Ryan Kilkenny Ryan , Jack Jack Ryan Tipperary 1949 Ryan , Jer Jer Ryan Tipperary 1887 Ryan , Joe Joe Ryan Kilkenny 1909 Ryan , John John Ryan Offaly 1998 Ryan , John John Ryan Galway 1980 Ryan , Johnny Johnny Ryan Tipperary 1937 Ryan , Matty Matty Ryan Tipperary 1900 Ryan , Matty Matty Ryan Tipperary 1945 Ryan , Michael Michael Ryan Tipperary 1991 Ryan , Miche\\u00e1l Miche\\u00e1l Ryan Tipperary 2001 Ryan , Mick Mick Ryan Tipperary 1930 Ryan , P.J. P.J. Ryan Tipperary 1971 Ryan , Paddy Paddy Ryan Laois 1915 Ryan , Pat Pat Ryan Dublin 1889 Ryan , Robbie Robbie Ryan Dublin 1938 Ryan , Roger Roger Ryan Tipperary 1971 Ryan , Tim Tim Ryan Tipperary 1951 Ryan , Tom Tom Ryan Limerick 1973 Ryan , Tom Tom Ryan Tipperary 1962 Ryan , Tom\\u00e1s Tom\\u00e1s Ryan Cork 1970 Ryng , James James Ryng Cork 1941 Saunders , Paddy Paddy Saunders Kilkenny 1904 Scallan , \\u00c9amonn \\u00c9amonn Scallan Wexford Scanlan , Phil Phil Scanlan Tipperary 1896 Shaloo , Jack Jack Shaloo Clare 1914 Shanahan , Tommy Tommy Shanahan Tipperary 1916 Sheehan , Colm Colm Sheehan Cork 1966 Sheehan , Connie Connie Sheehan Cork 1919 Sheehan , Mikey Mikey Sheehan Cork 1892 Shinny , Tom Tom Shinny Limerick 1934 Shortall , Mick Mick Shortall Kilkenny 1909 Shortle , Jim Jim Shortle Wexford 1910 Silke , Se\\u00e1n Se\\u00e1n Silke Galway 1980 Small , Willie Willie Small Dublin 1924 Spain , W.J. W.J. Spain Dublin 1889 Won a football medal in 1887 Spellissey , Jim Jim Spellissey Clare 1914 Stakelum , Conor Conor Stakelum Tipperary 1991 Staples , Vinny Vinny Staples Wexford 1968 Stapleton , Jim Jim Stapleton Tipperary 1887 Stapleton , Paddy Paddy Stapleton Tipperary Stapleton , Tom Tom Stapleton Tipperary 1887 Stokes , Dick Dick Stokes Limerick 1940 Storey , Martin Martin Storey Wexford Stuart , Charlie Charlie Stuart Dublin 1917 Sullivan , Dan Dan Sullivan Cork 1890 Sullivan , Ger Ger Sullivan Cork 1890 Sutton , John John Sutton Kilkenny 1957 Taaffe , \\u00c9amonn \\u00c9amonn Taaffe Clare 1995 Teehan , John John Teehan Kilkenny 1967 Teehan , Ned Ned Teehan Kilkenny 1905 Teehan , Tom Tom Teehan Dublin 1938 Tobin , Charlie Charlie Tobin Cork 1942 Tobin , Dick Dick Tobin Kilkenny 1922 Tobin , Jimmy Jimmy Tobin Kilkenny 1922 Toomey , Tom Tom Toomey Cork 1890 Treacy , Martin Martin Treacy Kilkenny 1963 Troy , Dan Dan Troy Limerick 1918 Wall , Tom Tom Wall Tipperary 1945 Wall , Willie Willie Wall Tipperary 1937 Walsh , Bill Bill Walsh Kilkenny 1957 Walsh , Dick Dick Walsh Tipperary 1916 Walsh , Frankie Frankie Walsh Waterford 1959 Walsh , Jim Jim Walsh Kilkenny 1957 Walsh , Liam Liam Walsh Kilkenny 1992 Walsh , Mick Mick Walsh Kilkenny 1957 Walsh , Tom Tom Walsh Kilkenny 1957 Walsh , Willie Willie Walsh Cork 1970 Walsh , John John Walsh Laois Dublin 1915 Walton , Tom Tom Walton Kilkenny 1947 Ware , Jim Jim Ware Waterford 1948 Waters , Mick Mick Waters Cork 1966 Whelan , Cha Cha Whelan Kilkenny 1963 Whelan , Donal Donal Whelan Waterford 1959 Whelan , S\\u00e9amus S\\u00e9amus Whelan Wexford 1968 Whelehan , Simon Simon Whelehan Offaly 1998 Wilson , Phil Phil Wilson Wexford 1968 Wynne , Pat Pat Wynne Kerry 1891 Young , David David Young Tipperary Farrell , Sean O \' ( John ) Sean O \' ( John ) Farrell Kilkenny 1933 Won a medal as a non-playing substitute"

In [134]:
len(lala)

58875

In [139]:
aa = Path('datasets/msmarco.parquet')

In [140]:
aa.stem

'msmarco'

In [141]:
returned_dict = {
             "id": [],
             "query": [],
             "passage_text": [],
        }
failed = []
# Open and iterate through the .jsonl file
with open(Path("translation_pipeline_test/msmarco_test_results.jsonl"), 'r') as file:
    for line in file:
        id_ = None
        try:
            data = json.loads(line)
            id_ = data[-1]['id']
            returned_data = data[1]['choices'][0]['message']['content']
            tranlation = json.loads(returned_data) # gpt message i.e. translation in this case
            # print(type(tranlation['passage_text']))
            if isinstance(tranlation['passage_text'], int) or isinstance(tranlation['query'], int) or isinstance(id_, int):
                print("AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA")
                print(id_)
            # tranlation = returned_data['translations']
            returned_dict['id'].append(id_)
            returned_dict['query'].append(tranlation['query'])
            returned_dict['passage_text'].append(tranlation['passage_text'])
        except Exception as e:
            failed.append({
                'id': id_,
                'exception': e
            })



In [145]:
def all_items_same_type(lst):
    if not lst:  # Handle the case where the list is empty
        return True
    first_type = type(lst[0])
    print(first_type)
    return all(isinstance(item, first_type) for item in lst)

# Example usage:
if not all_items_same_type(returned_dict['passage_text']):
    print("Not all items are of the same type")
else:
    print("All items are of the same type")

# Example usage:
if not all_items_same_type(returned_dict['id']):
    print("Not all items are of the same type")
else:
    print("All items are of the same type")

# Example usage:
if not all_items_same_type(returned_dict['query']):
    print("Not all items are of the same type")
else:
    print("All items are of the same type")

<class 'list'>
All items are of the same type
<class 'str'>
All items are of the same type
<class 'str'>
All items are of the same type


In [158]:
dataset = pd.DataFrame(returned_dict)
# dataset['id'] = dataset['id'].astype(str)
# dataset['query'] = dataset['query'].astype(str)
dataset['passage_text'] = dataset['passage_text'].apply(lambda x: np.array(x, dtype=str))

dataset.to_parquet(Path("datasets/tralala.parquet"), engine='pyarrow')

ArrowTypeError: ("Expected bytes, got a 'int' object", 'Conversion failed for column passage_text with type object')